![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Clinical Named Entity Recognition Models

Please make sure that your cluster is setup properly according to https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-spark-nlp-for-healthcare-on-databricks

**NER Model Implementation in Spark NLP**

  The deep neural network architecture for NER model in
Spark NLP is BiLSTM-CNN-Char framework. a slightly modified version of the architecture proposed by Jason PC Chiu and Eric Nichols ([Named Entity Recognition with Bidirectional LSTM-CNNs
](https://arxiv.org/abs/1511.08308)). It is a neural network architecture that
automatically detects word and character-level features using a
hybrid bidirectional LSTM and CNN architecture, eliminating
the need for most feature engineering steps.

  In the original framework, the CNN extracts a fixed length
feature vector from character-level features. For each word,
these vectors are concatenated and fed to the BLSTM network
and then to the output layers. They employed a stacked
bi-directional recurrent neural network with long short-term
memory units to transform word features into named entity
tag scores. The extracted features of each word are fed into a
forward LSTM network and a backward LSTM network. The
output of each network at each time step is decoded by a linear
layer and a log-softmax layer into log-probabilities for each tag
category. These two vectors are then simply added together to
produce the final output. In the architecture of the proposed framework in the original paper, 50-dimensional pretrained word
embeddings is used for word features, 25-dimension character
embeddings is used for char features, and capitalization features
(allCaps, upperInitial, lowercase, mixedCaps, noinfo) are used
for case features.

## Blogposts and videos:

- [How to Setup Spark NLP for HEALTHCARE on UBUNTU - Video](https://www.youtube.com/watch?v=yKnF-_oz0GE)

- [Named Entity Recognition (NER) with BERT in Spark NLP](https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77)

- [State of the art Clinical Named Entity Recognition in Spark NLP - Youtube](https://www.youtube.com/watch?v=YM-e4eOiQ34)

- [Named Entity Recognition for Healthcare with SparkNLP NerDL and NerCRF](https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571)

- [Named Entity Recognition for Clinical Text](https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180)

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd

import nlu
import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 5.0.1
sparknlp_jsl.version : 5.0.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

# Clinical NER Pipeline (with pretrained models)

## Clinical NER Models
|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [jsl_ner_wip_clinical](https://nlp.johnsnowlabs.com/2021/03/31/jsl_ner_wip_clinical_en.html)  | 2| [jsl_ner_wip_greedy_clinical](https://nlp.johnsnowlabs.com/2021/03/31/jsl_ner_wip_greedy_clinical_en.html)  | 3| [jsl_ner_wip_modifier_clinical](https://nlp.johnsnowlabs.com/2021/04/01/jsl_ner_wip_modifier_clinical_en.html)  | 4| [jsl_rd_ner_wip_greedy_clinical](https://nlp.johnsnowlabs.com/2021/04/01/jsl_rd_ner_wip_greedy_clinical_en.html)  |
| 5| [ner_abbreviation_clinical](https://nlp.johnsnowlabs.com/2021/12/30/ner_abbreviation_clinical_en.html)  | 6| [ner_abbreviation_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/05/12/ner_abbreviation_emb_clinical_large_en.html)  | 7| [ner_abbreviation_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/05/12/ner_abbreviation_emb_clinical_medium_en.html)  | 8| [ner_ade_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinical_en.html)  |
| 9| [ner_ade_clinicalbert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinicalbert_en.html)  | 10| [ner_ade_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/05/21/ner_ade_emb_clinical_large_en.html)  | 11| [ner_ade_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/05/21/ner_ade_emb_clinical_medium_en.html)  | 12| [ner_ade_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_healthcare_en.html)  |
| 13| [ner_anatomy](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_en.html)  | 14| [ner_anatomy_coarse](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_coarse_en.html)  | 15| [ner_anatomy_coarse_en](https://nlp.johnsnowlabs.com/2020/11/04/ner_anatomy_coarse_en.html)  | 16| [ner_anatomy_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/05/15/ner_anatomy_emb_clinical_large_en.html)  |
| 17| [ner_anatomy_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/05/15/ner_anatomy_emb_clinical_medium_en.html)  | 18| [ner_anatomy_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_anatomy_en.html)  | 19| [ner_aspect_based_sentiment](https://nlp.johnsnowlabs.com/2021/03/31/ner_aspect_based_sentiment_en.html)  | 20| [ner_bacterial_species](https://nlp.johnsnowlabs.com/2021/04/01/ner_bacterial_species_en.html)  |
| 21| [ner_bacterial_species_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/05/23/ner_bacterial_species_emb_clinical_large_en.html)  | 22| [ner_bacterial_species_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/05/23/ner_bacterial_species_emb_clinical_medium_en.html)  | 23| [ner_biomarker](https://nlp.johnsnowlabs.com/2021/11/26/ner_biomarker_en.html)  | 24| [ner_biomedical_bc2gm](https://nlp.johnsnowlabs.com/2022/05/11/ner_biomedical_bc2gm_en_2_4.html)  |
| 25| [ner_bionlp](https://nlp.johnsnowlabs.com/2021/03/31/ner_bionlp_en.html)  | 26| [ner_bionlp_en](https://nlp.johnsnowlabs.com/2020/01/30/ner_bionlp_en.html)  | 27| [ner_cancer_genetics](https://nlp.johnsnowlabs.com/2021/03/31/ner_cancer_genetics_en.html)  | 28| [ner_cellular](https://nlp.johnsnowlabs.com/2021/03/31/ner_cellular_en.html)  |
| 29| [ner_cellular_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/05/24/ner_cellular_emb_clinical_large_en.html)  | 30| [ner_cellular_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/05/24/ner_cellular_emb_clinical_medium_en.html)  | 31| [ner_cellular_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_cellular_en.html)  | 32| [ner_chemd_clinical](https://nlp.johnsnowlabs.com/2021/11/04/ner_chemd_clinical_en.html)  |
| 33| [ner_chemicals](https://nlp.johnsnowlabs.com/2021/04/01/ner_chemicals_en.html)  | 34| [ner_chemicals_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/02/ner_chemicals_emb_clinical_large_en.html)  | 35| [ner_chemicals_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/02/ner_chemicals_emb_clinical_medium_en.html)  | 36| [ner_chemprot_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_chemprot_clinical_en.html)  |
| 37| [ner_chexpert](https://nlp.johnsnowlabs.com/2021/09/30/ner_chexpert_en.html)  | 38| [ner_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_clinical_en.html)  | 39| [ner_clinical_en](https://nlp.johnsnowlabs.com/2020/01/30/ner_clinical_en.html)  | 40| [ner_clinical_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_clinical_large_en.html)  |
| 41| [ner_clinical_large_en](https://nlp.johnsnowlabs.com/2020/05/23/ner_clinical_large_en.html)  | 42| [ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/06/22/ner_clinical_trials_abstracts_en_3_0.html)  | 43| [ner_covid_trials](https://nlp.johnsnowlabs.com/2022/10/19/ner_covid_trials_en.html)  | 44| [ner_crf](https://nlp.johnsnowlabs.com/2020/01/28/ner_crf_en.html)  |
| 45| [ner_deid_augmented](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_augmented_en.html)  | 46| [ner_deid_enriched](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_enriched_en.html)  | 47| [ner_deid_generic_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_generic_glove_en.html)  | 48| [ner_deid_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_large_en.html)  |
| 49| [ner_deid_large_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/04/12/ner_deid_large_emb_clinical_large_en.html)  | 50| [ner_deid_large_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/04/12/ner_deid_large_emb_clinical_medium_en.html)  | 51| [ner_deid_sd](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_en.html)  | 52| [ner_deid_sd_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_sd_large_en.html)  |
| 53| [ner_deid_subentity_augmented](https://nlp.johnsnowlabs.com/2021/09/03/ner_deid_subentity_augmented_en.html)  | 54| [ner_deid_subentity_augmented_i2b2](https://nlp.johnsnowlabs.com/2021/11/29/ner_deid_subentity_augmented_i2b2_en.html)  | 55| [ner_deid_subentity_glove](https://nlp.johnsnowlabs.com/2021/06/06/ner_deid_subentity_glove_en.html)  | 56| [ner_deid_synthetic](https://nlp.johnsnowlabs.com/2021/03/31/ner_deid_synthetic_en.html)  |
| 57| [ner_deidentify_dl](https://nlp.johnsnowlabs.com/2021/03/31/ner_deidentify_dl_en.html)  | 58| [ner_demographic_extended_healthcare](https://nlp.johnsnowlabs.com/2023/06/08/ner_demographic_extended_healthcare_en.html)  | 59| [ner_diseases](https://nlp.johnsnowlabs.com/2021/03/31/ner_diseases_en.html)  | 60| [ner_diseases_en](https://nlp.johnsnowlabs.com/2020/03/25/ner_diseases_en.html)  |
| 61| [ner_diseases_large](https://nlp.johnsnowlabs.com/2021/04/01/ner_diseases_large_en.html)  | 62| [ner_drugprot_clinical](https://nlp.johnsnowlabs.com/2021/12/20/ner_drugprot_clinical_en.html)  | 63| [ner_drugs](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_en.html)  | 64| [ner_drugs_en](https://nlp.johnsnowlabs.com/2020/03/25/ner_drugs_en.html)  |
| 65| [ner_drugs_greedy](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_greedy_en.html)  | 66| [ner_drugs_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_drugs_large_en.html)  | 67| [ner_drugs_large_en](https://nlp.johnsnowlabs.com/2021/01/29/ner_drugs_large_en.html)  | 68| [ner_eu_clinical_case](https://nlp.johnsnowlabs.com/2023/01/25/ner_eu_clinical_case_en.html)  |
| 69| [ner_eu_clinical_condition](https://nlp.johnsnowlabs.com/2023/02/06/ner_eu_clinical_condition_en.html)  | 70| [ner_events_admission_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_events_admission_clinical_en.html)  | 71| [ner_events_clinical](https://nlp.johnsnowlabs.com/2021/03/31/ner_events_clinical_en.html)  | 72| [ner_events_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_events_healthcare_en.html)  |
| 73| [ner_financial_contract](https://nlp.johnsnowlabs.com/2021/04/01/ner_financial_contract_en.html)  | 74| [ner_genetic_variants](https://nlp.johnsnowlabs.com/2021/06/25/ner_genetic_variants_en.html)  | 75| [ner_healthcare](https://nlp.johnsnowlabs.com/2021/04/21/ner_healthcare_en.html)  | 76| [ner_healthcare_en](https://nlp.johnsnowlabs.com/2020/03/26/ner_healthcare_en.html)  |
| 77| [ner_human_phenotype_gene_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_human_phenotype_gene_clinical_en.html)  | 78| [ner_human_phenotype_go_clinical](https://nlp.johnsnowlabs.com/2020/09/21/ner_human_phenotype_go_clinical_en.html)  | 79| [ner_jsl](https://nlp.johnsnowlabs.com/2022/10/19/ner_jsl_en.html)  | 80| [ner_jsl_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/04/12/ner_jsl_emb_clinical_large_en.html)  |
| 81| [ner_jsl_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/04/12/ner_jsl_emb_clinical_medium_en.html)  | 82| [ner_jsl_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_jsl_en.html)  | 83| [ner_jsl_enriched](https://nlp.johnsnowlabs.com/2021/10/22/ner_jsl_enriched_en.html)  | 84| [ner_jsl_enriched_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_jsl_enriched_en.html)  |
| 85| [ner_jsl_greedy](https://nlp.johnsnowlabs.com/2021/06/24/ner_jsl_greedy_en.html)  | 86| [ner_jsl_limited_80p_for_benchmarks](https://nlp.johnsnowlabs.com/2023/04/02/ner_jsl_limited_80p_for_benchmarks_en.html)  | 87| [ner_jsl_slim](https://nlp.johnsnowlabs.com/2021/08/13/ner_jsl_slim_en.html)  | 88| [ner_living_species](https://nlp.johnsnowlabs.com/2022/06/22/ner_living_species_en_3_0.html)  |
| 89| [ner_measurements_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_measurements_clinical_en.html)  | 90| [ner_medmentions_coarse](https://nlp.johnsnowlabs.com/2021/04/01/ner_medmentions_coarse_en.html)  | 91| [ner_nature_nero_clinical](https://nlp.johnsnowlabs.com/2022/02/08/ner_nature_nero_clinical_en.html)  | 92| [ner_nihss](https://nlp.johnsnowlabs.com/2021/11/15/ner_nihss_en.html)  |
| 93| [ner_oncology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_en.html)  | 94| [ner_oncology_anatomy_general](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_anatomy_general_en.html)  | 95| [ner_oncology_anatomy_general_healthcare](https://nlp.johnsnowlabs.com/2023/01/11/ner_oncology_anatomy_general_healthcare_en.html)  | 96| [ner_oncology_anatomy_granular](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_anatomy_granular_en.html)  |
| 97| [ner_oncology_biomarker](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_biomarker_en.html)  | 98| [ner_oncology_biomarker_healthcare](https://nlp.johnsnowlabs.com/2023/01/11/ner_oncology_biomarker_healthcare_en.html)  | 99| [ner_oncology_demographics](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_demographics_en.html)  | 100| [ner_oncology_diagnosis](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_diagnosis_en.html)  |
| 101| [ner_oncology_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/04/12/ner_oncology_emb_clinical_large_en.html)  | 102| [ner_oncology_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/04/12/ner_oncology_emb_clinical_medium_en.html)  | 103| [ner_oncology_limited_80p_for_benchmarks](https://nlp.johnsnowlabs.com/2023/04/03/ner_oncology_limited_80p_for_benchmarks_en.html)  | 104| [ner_oncology_posology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_posology_en.html)  |
| 105| [ner_oncology_response_to_treatment](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_response_to_treatment_en.html)  | 106| [ner_oncology_test](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_test_en.html)  | 107| [ner_oncology_therapy](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_therapy_en.html)  | 108| [ner_oncology_tnm](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_tnm_en.html)  |
| 109| [ner_oncology_unspecific_posology](https://nlp.johnsnowlabs.com/2022/10/25/ner_oncology_unspecific_posology_en.html)  | 110| [ner_oncology_unspecific_posology_healthcare](https://nlp.johnsnowlabs.com/2023/01/11/ner_oncology_unspecific_posology_healthcare_en.html)  | 111| [ner_pathogen](https://nlp.johnsnowlabs.com/2022/06/28/ner_pathogen_en_3_0.html)  | 112| [ner_posology](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_en.html)  |
| 113| [ner_posology_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/04/12/ner_posology_emb_clinical_large_en.html)  | 114| [ner_posology_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/04/12/ner_posology_emb_clinical_medium_en.html)  | 115| [ner_posology_en](https://nlp.johnsnowlabs.com/2020/04/15/ner_posology_en.html)  | 116| [ner_posology_experimental](https://nlp.johnsnowlabs.com/2021/09/01/ner_posology_experimental_en.html)  |
| 117| [ner_posology_greedy](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_greedy_en.html)  | 118| [ner_posology_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_healthcare_en.html)  | 119| [ner_posology_large](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_large_en.html)  | 120| [ner_posology_small](https://nlp.johnsnowlabs.com/2021/03/31/ner_posology_small_en.html)  |
| 121| [ner_radiology](https://nlp.johnsnowlabs.com/2021/03/31/ner_radiology_en.html)  | 122| [ner_radiology_wip_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_radiology_wip_clinical_en.html)  | 123| [ner_risk_factors](https://nlp.johnsnowlabs.com/2021/03/31/ner_risk_factors_en.html)  | 124| [ner_risk_factors_en](https://nlp.johnsnowlabs.com/2020/04/22/ner_risk_factors_en.html)  |
| 125| [ner_sdoh](https://nlp.johnsnowlabs.com/2023/06/13/ner_sdoh_en.html)  | 126| [ner_sdoh_mentions](https://nlp.johnsnowlabs.com/2022/12/18/ner_sdoh_mentions_en.html)  | 127| [ner_supplement_clinical](https://nlp.johnsnowlabs.com/2022/02/01/ner_supplement_clinical_en.html)  | 128| [ner_vop](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_en.html)  |
| 129| [ner_vop_anatomy](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_anatomy_en.html)  | 130| [ner_vop_anatomy_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_anatomy_emb_clinical_large_en.html)  | 131| [ner_vop_anatomy_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/07/ner_vop_anatomy_emb_clinical_medium_en.html)  | 132| [ner_vop_clinical_dept](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_clinical_dept_en.html)  |
| 133| [ner_vop_clinical_dept_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_clinical_dept_emb_clinical_large_en.html)  | 134| [ner_vop_clinical_dept_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_clinical_dept_emb_clinical_medium_en.html)  | 135| [ner_vop_demographic](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_demographic_en.html)  | 136| [ner_vop_demographic_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_demographic_emb_clinical_large_en.html)  |
| 137| [ner_vop_demographic_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_demographic_emb_clinical_medium_en.html)  | 138| [ner_vop_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_emb_clinical_large_en.html)  | 139| [ner_vop_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_emb_clinical_medium_en.html)  | 140| [ner_vop_problem](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_problem_en.html)  |
| 141| [ner_vop_problem_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_problem_emb_clinical_large_en.html)  | 142| [ner_vop_problem_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_problem_emb_clinical_medium_en.html)  | 143| [ner_vop_problem_reduced](https://nlp.johnsnowlabs.com/2023/06/07/ner_vop_problem_reduced_en.html)  | 144| [ner_vop_problem_reduced_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/07/ner_vop_problem_reduced_emb_clinical_large_en.html)  |
| 145| [ner_vop_problem_reduced_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/07/ner_vop_problem_reduced_emb_clinical_medium_en.html)  | 146| [ner_vop_temporal](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_temporal_en.html)  | 147| [ner_vop_temporal_emb_clinical_large_final](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_temporal_emb_clinical_large_final_en.html)  | 148| [ner_vop_temporal_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_temporal_emb_clinical_medium_en.html)  |
| 149| [ner_vop_test](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_test_en.html)  | 150| [ner_vop_test_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_test_emb_clinical_large_en.html)  | 151| [ner_vop_test_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_test_emb_clinical_medium_en.html)  | 152| [ner_vop_treatment](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_treatment_en.html)  |
| 153| [ner_vop_treatment_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_treatment_emb_clinical_large_en.html)  | 154| [ner_vop_treatment_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/06/06/ner_vop_treatment_emb_clinical_medium_en.html)  | 155| [ner_vop_wip_emb_clinical_large](https://nlp.johnsnowlabs.com/2023/05/19/ner_vop_wip_embeddings_clinical_large_en.html)  | 156| [ner_vop_wip_emb_clinical_medium](https://nlp.johnsnowlabs.com/2023/05/19/ner_vop_wip_embeddings_clinical_medium_en.html)  |
| 157| [nerdl_tumour_demo](https://nlp.johnsnowlabs.com/2021/04/01/nerdl_tumour_demo_en.html)  |



- **Multilingual Clinical NER Models**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [ner_clinical](https://nlp.johnsnowlabs.com/2023/05/08/ner_clinical_de.html)  | 2| [ner_clinical_bert](https://nlp.johnsnowlabs.com/2022/11/22/ner_clinical_bert_ro.html)  | 3| [ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/08/12/ner_clinical_trials_abstracts_es_3_0.html)  | 4| [ner_deid_generic](https://nlp.johnsnowlabs.com/2023/05/30/ner_deid_generic_ar.html)  |
| 5| [ner_deid_generic_bert](https://nlp.johnsnowlabs.com/2022/11/22/ner_deid_generic_bert_ro.html)  | 6| [ner_deid_subentity](https://nlp.johnsnowlabs.com/2023/05/31/ner_deid_subentity_ar.html)  | 7| [ner_deid_subentity_bert](https://nlp.johnsnowlabs.com/2022/06/27/ner_deid_subentity_bert_ro_3_0.html)  | 8| [ner_diag_proc](https://nlp.johnsnowlabs.com/2021/03/31/ner_diag_proc_es.html)  |
| 9| [ner_eu_clinical_case](https://nlp.johnsnowlabs.com/2023/02/02/ner_eu_clinical_case_eu.html)  | 10| [ner_eu_clinical_condition](https://nlp.johnsnowlabs.com/2023/02/06/ner_eu_clinical_condition_es.html)  | 11| [ner_healthcare](https://nlp.johnsnowlabs.com/2020/09/28/ner_healthcare_de.html)  | 12| [ner_healthcare_slim](https://nlp.johnsnowlabs.com/2021/04/01/ner_healthcare_slim_de.html)  |
| 13| [ner_living_species](https://nlp.johnsnowlabs.com/2022/06/23/ner_living_species_it_3_0.html)  | 14| [ner_living_species_300](https://nlp.johnsnowlabs.com/2022/11/22/ner_living_species_300_es.html)  | 15| [ner_living_species_bert](https://nlp.johnsnowlabs.com/2022/06/23/ner_living_species_bert_ro_3_0.html)  | 16| [ner_living_species_roberta](https://nlp.johnsnowlabs.com/2022/06/22/ner_living_species_roberta_pt_3_0.html)  |
| 17| [ner_negation_uncertainty](https://nlp.johnsnowlabs.com/2022/08/13/ner_negation_uncertainty_es_3_0.html)  | 18| [ner_neoplasms](https://nlp.johnsnowlabs.com/2021/03/31/ner_neoplasms_es.html)  | 19| [ner_pharmacology](https://nlp.johnsnowlabs.com/2022/08/13/ner_pharmacology_es_3_0.html)  | 20| [ner_traffic](https://nlp.johnsnowlabs.com/2021/04/01/ner_traffic_de.html)  |

- **BioBert NER Models**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [jsl_ner_wip_greedy_biobert](https://nlp.johnsnowlabs.com/2021/07/26/jsl_ner_wip_greedy_biobert_en.html)  | 2| [jsl_rd_ner_wip_greedy_biobert](https://nlp.johnsnowlabs.com/2021/07/26/jsl_rd_ner_wip_greedy_biobert_en.html)  | 3| [ner_ade_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_biobert_en.html)  | 4| [ner_anatomy_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_anatomy_biobert_en.html)  |
| 5| [ner_anatomy_coarse_biobert](https://nlp.johnsnowlabs.com/2021/03/31/ner_anatomy_coarse_biobert_en.html)  | 6| [ner_anatomy_coarse_biobert_en](https://nlp.johnsnowlabs.com/2020/11/04/ner_anatomy_coarse_biobert_en.html)  | 7| [ner_bionlp_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_bionlp_biobert_en.html)  | 8| [ner_cellular_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_cellular_biobert_en.html)  |
| 9| [ner_chemprot_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_chemprot_biobert_en.html)  | 10| [ner_clinical_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_clinical_biobert_en.html)  | 11| [ner_deid_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_biobert_en.html)  | 12| [ner_deid_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_deid_enriched_biobert_en.html)  |
| 13| [ner_diseases_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_diseases_biobert_en.html)  | 14| [ner_events_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_events_biobert_en.html)  | 15| [ner_human_phenotype_gene_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_human_phenotype_gene_biobert_en.html)  | 16| [ner_human_phenotype_go_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_human_phenotype_go_biobert_en.html)  |
| 17| [ner_jsl_biobert](https://nlp.johnsnowlabs.com/2021/09/05/ner_jsl_biobert_en.html)  | 18| [ner_jsl_enriched_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_jsl_enriched_biobert_en.html)  | 19| [ner_jsl_greedy_biobert](https://nlp.johnsnowlabs.com/2021/08/13/ner_jsl_greedy_biobert_en.html)  | 20| [ner_living_species_biobert](https://nlp.johnsnowlabs.com/2022/06/22/ner_living_species_biobert_en_3_0.html)  |
| 21| [ner_posology_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_biobert_en.html)  | 22| [ner_posology_large_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_posology_large_biobert_en.html)  | 23| [ner_profiling_biobert](https://nlp.johnsnowlabs.com/2023/06/13/ner_profiling_biobert_en.html)  | 24| [ner_risk_factors_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_risk_factors_biobert_en.html)  |




- **BertForTokenClassification Clinical NER models**

|index|model|index|model|index|model|index|model|
|-----:|:-----|-----:|:-----|-----:|:-----|-----:|:-----|
| 1| [bert_token_classifier_ade_tweet_binary](https://nlp.johnsnowlabs.com/2022/07/29/bert_token_classifier_ade_tweet_binary_en_3_0.html)  | 2| [bert_token_classifier_drug_development_trials](https://nlp.johnsnowlabs.com/2022/03/22/bert_token_classifier_drug_development_trials_en_3_0.html)  | 3| [bert_token_classifier_ner_ade](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_ade_en.html)  | 4| [bert_token_classifier_ner_ade_binary](https://nlp.johnsnowlabs.com/2022/07/27/bert_token_classifier_ner_ade_binary_en_3_0.html)  |
| 5| [bert_token_classifier_ner_anatem](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_anatem_en_3_0.html)  | 6| [bert_token_classifier_ner_anatomy](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_anatomy_en.html)  | 7| [bert_token_classifier_ner_bacteria](https://nlp.johnsnowlabs.com/2021/09/30/bert_token_classifier_ner_bacteria_en.html)  | 8| [bert_token_classifier_ner_bc2gm_gene](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc2gm_gene_en_3_0.html)  |
| 9| [bert_token_classifier_ner_bc4chemd_chemicals](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc4chemd_chemicals_en_3_0.html)  | 10| [bert_token_classifier_ner_bc5cdr_chemicals](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc5cdr_chemicals_en_3_0.html)  | 11| [bert_token_classifier_ner_bc5cdr_disease](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_bc5cdr_disease_en_3_0.html)  | 12| [bert_token_classifier_ner_bionlp](https://nlp.johnsnowlabs.com/2021/11/03/bert_token_classifier_ner_bionlp_en.html)  |
| 13| [bert_token_classifier_ner_cellular](https://nlp.johnsnowlabs.com/2021/11/03/bert_token_classifier_ner_cellular_en.html)  | 14| [bert_token_classifier_ner_chemicals](https://nlp.johnsnowlabs.com/2021/10/19/bert_token_classifier_ner_chemicals_en.html)  | 15| [bert_token_classifier_ner_chemprot](https://nlp.johnsnowlabs.com/2021/10/19/bert_token_classifier_ner_chemprot_en.html)  | 16| [bert_token_classifier_ner_clinical](https://nlp.johnsnowlabs.com/2022/01/06/bert_token_classifier_ner_clinical_en.html)  |
| 17| [bert_token_classifier_ner_clinical_trials_abstracts](https://nlp.johnsnowlabs.com/2022/06/29/bert_token_classifier_ner_clinical_trials_abstracts_en_3_0.html)  | 18| [bert_token_classifier_ner_deid](https://nlp.johnsnowlabs.com/2021/09/13/bert_token_classifier_ner_deid_en.html)  | 19| [bert_token_classifier_ner_drugs](https://nlp.johnsnowlabs.com/2021/09/20/bert_token_classifier_ner_drugs_en.html)  | 20| [bert_token_classifier_ner_jnlpba_cellular](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_jnlpba_cellular_en_3_0.html)  |
| 21| [bert_token_classifier_ner_jsl](https://nlp.johnsnowlabs.com/2021/09/16/bert_token_classifier_ner_jsl_en.html)  | 22| [bert_token_classifier_ner_jsl_slim](https://nlp.johnsnowlabs.com/2021/09/24/bert_token_classifier_ner_jsl_slim_en.html)  | 23| [bert_token_classifier_ner_linnaeus_species](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_linnaeus_species_en_3_0.html)  | 24| [bert_token_classifier_ner_living_species](https://nlp.johnsnowlabs.com/2022/06/26/bert_token_classifier_ner_living_species_en_3_0.html)  |
| 25| [bert_token_classifier_ner_ncbi_disease](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_ncbi_disease_en_3_0.html)  | 26| [bert_token_classifier_ner_pathogen](https://nlp.johnsnowlabs.com/2022/07/28/bert_token_classifier_ner_pathogen_en_3_0.html)  | 27| [bert_token_classifier_ner_species](https://nlp.johnsnowlabs.com/2022/07/25/bert_token_classifier_ner_species_en_3_0.html)  | 28| [bert_token_classifier_ner_supplement](https://nlp.johnsnowlabs.com/2022/02/09/bert_token_classifier_ner_supplement_en.html)  |






**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?task=Named+Entity+Recognition&edition=Spark+NLP+for+Healthcare)**

**Also, you can print the list of clinical pretrained models/pipelines and annotators with one-line code:**

In [0]:
from sparknlp_jsl.pretrained import InternalResourceDownloader

# pipelines = InternalResourceDownloader.returnPrivatePipelines()
ner_models = InternalResourceDownloader.returnPrivateModels("MedicalNerModel")
ner_models

Out[2]: [['nerdl_tumour_demo', 'en', '1.7.3'],
 ['nerdl_tumour_demo', 'en', '1.8.0'],
 ['nerdl_tumour_demo', 'en', '2.0.2'],
 ['ner_healthcare', 'en', '2.4.4'],
 ['ner_radiology', 'en', '2.7.0'],
 ['ner_deid_augmented', 'en', '2.7.1'],
 ['ner_deidentify_dl', 'en', '2.7.2'],
 ['ner_events_admission_clinical', 'en', '2.7.4'],
 ['ner_clinical', 'en', '3.0.0'],
 ['ner_radiology', 'en', '3.0.0'],
 ['ner_bionlp', 'en', '3.0.0'],
 ['ner_posology', 'en', '3.0.0'],
 ['ner_healthcare', 'de', '3.0.0'],
 ['ner_deid_augmented', 'en', '3.0.0'],
 ['ner_anatomy', 'en', '3.0.0'],
 ['ner_risk_factors', 'en', '3.0.0'],
 ['ner_chemprot_clinical', 'en', '3.0.0'],
 ['ner_posology_small', 'en', '3.0.0'],
 ['ner_diag_proc', 'es', '3.0.0'],
 ['ner_posology_greedy', 'en', '3.0.0'],
 ['ner_deid_enriched', 'en', '3.0.0'],
 ['ner_drugs_greedy', 'en', '3.0.0'],
 ['jsl_ner_wip_clinical', 'en', '3.0.0'],
 ['ner_posology_large', 'en', '3.0.0'],
 ['jsl_ner_wip_greedy_clinical', 'en', '3.0.0'],
 ['ner_clinical_large', 'en', '3.0.0'],
 ['ner_diseases', 'en', '3.0.0'],
 ['ner_aspect_based_sentiment', 'en', '3.0.0'],
 ['ner_cellular', 'en', '3.0.0'],
 ['ner_drugs', 'en', '3.0.0'],
 ['ner_jsl', 'en', '3.0.0'],
 ['ner_anatomy_coarse_biobert', 'en', '3.0.0'],
 ['ner_cancer_genetics', 'en', '3.0.0'],
 ['ner_drugs_large', 'en', '3.0.0'],
 ['ner_events_admission_clinical', 'en', '3.0.0'],
 ['ner_deidentify_dl', 'en', '3.0.0'],
 ['ner_deid_synthetic', 'en', '3.0.0'],
 ['ner_jsl_enriched', 'en', '3.0.0'],
 ['ner_deid_large', 'en', '3.0.0'],
 ['ner_human_phenotype_gene_clinical', 'en', '3.0.0'],
 ['ner_human_phenotype_go_clinical', 'en', '3.0.0'],
 ['ner_events_clinical', 'en', '3.0.0'],
 ['ner_legal', 'de', '3.0.0'],
 ['ner_anatomy_coarse', 'en', '3.0.0'],
 ['ner_traffic', 'de', '3.0.0'],
 ['ner_bionlp_biobert', 'en', '3.0.0'],
 ['ner_deid_sd_large', 'en', '3.0.0'],
 ['ner_healthcare_slim', 'de', '3.0.0'],
 ['ner_risk_factors_biobert', 'en', '3.0.0'],
 ['ner_posology_healthcare', 'en', '3.0.0'],
 ['ner_diseases_large', 'en', '3.0.0'],
 ['ner_ade_biobert', 'en', '3.0.0'],
 ['ner_jsl_enriched_biobert', 'en', '3.0.0'],
 ['ner_ade_healthcare', 'en', '3.0.0'],
 ['ner_deid_sd', 'en', '3.0.0'],
 ['ner_events_healthcare', 'en', '3.0.0'],
 ['ner_financial_contract', 'en', '3.0.0'],
 ['ner_chemprot_biobert', 'en', '3.0.0'],
 ['ner_ade_clinicalbert', 'en', '3.0.0'],
 ['ner_posology_large_biobert', 'en', '3.0.0'],
 ['ner_radiology_wip_clinical', 'en', '3.0.0'],
 ['ner_jsl_biobert', 'en', '3.0.0'],
 ['ner_clinical_biobert', 'en', '3.0.0'],
 ['ner_deid_enriched_biobert', 'en', '3.0.0'],
 ['ner_posology_biobert', 'en', '3.0.0'],
 ['ner_cellular_biobert', 'en', '3.0.0'],
 ['jsl_ner_wip_modifier_clinical', 'en', '3.0.0'],
 ['ner_measurements_clinical', 'en', '3.0.0'],
 ['ner_medmentions_coarse', 'en', '3.0.0'],
 ['ner_chemicals', 'en', '3.0.0'],
 ['ner_events_biobert', 'en', '3.0.0'],
 ['ner_bacterial_species', 'en', '3.0.0'],
 ['ner_diseases_biobert', 'en', '3.0.0'],
 ['ner_human_phenotype_gene_biobert', 'en', '3.0.0'],
 ['ner_deid_biobert', 'en', '3.0.0'],
 ['nerdl_tumour_demo', 'en', '3.0.0'],
 ['ner_human_phenotype_go_biobert', 'en', '3.0.0'],
 ['ner_anatomy_biobert', 'en', '3.0.0'],
 ['ner_ade_clinical', 'en', '3.0.0'],
 ['jsl_rd_ner_wip_greedy_clinical', 'en', '3.0.0'],
 ['ner_healthcare', 'en', '3.0.0'],
 ['ner_deid_generic_augmented', 'en', '3.0.3'],
 ['ner_deid_subentity_augmented', 'en', '3.0.3'],
 ['ner_deid_generic_glove', 'en', '3.0.4'],
 ['ner_deid_subentity_glove', 'en', '3.0.4'],
 ['ner_jsl', 'en', '3.1.0'],
 ['ner_genetic_variants', 'en', '3.1.0'],
 ['ner_deid_subentity_augmented', 'en', '3.1.0'],
 ['ner_deid_generic_augmented', 'en', '3.1.0'],
 ['lener_bert_base', 'pt', '3.1.2'],
 ['cantemist_scielowiki', 'es', '3.1.2'],
 ['jsl_ner_wip_greedy_biobert', 'en', '3.1.3'],
 ['jsl_rd_ner_wip_greedy_biobert', 'en', '3.1.3'],
 ['lener_bert_large', 'pt', '3.1.3'],
 ['lener_bert_base', 'pt', '3.1.3'],
 ['meddroprof_scielowiki', 'es', '3.1.3'],
 ['ner_jsl_greedy_biobert', 'en', '3.

In [0]:
from sparknlp_jsl.compatibility import Compatibility 

compatibility = Compatibility(spark)

models = compatibility.findVersion('ner') 

models_df = pd.DataFrame([dict(x) for x in list(models)])
models_df

Out[3]:

,name,sparkVersion,version,language,date,readyToUse
0,nerdl_tumour_demo,2,1.7.3,en,2018-12-19T16:52:37.735,true
1,nercrf_tumour_demo,2,1.7.3,en,2018-12-19T17:23:53.776,true
2,nerdl_tumour_demo,2.4,1.8.0,en,2018-12-22T04:21:25.574,true
3,nercrf_tumour_demo,2.4,1.8.0,en,2018-12-22T04:46:26.992,true
4,nercrf_tumour_demo,2.4,2.0.2,en,2019-05-21T18:23:13.066,true
...,...,...,...,...,...,...
1593,ner_ade_clinical_langtest,3.0,5.0.0,en,2023-07-31T17:41:46.693,true
1594,ner_sdoh_langtest,3.0,5.0.0,en,2023-07-31T10:34:18.986,true
1595,ner_jsl_langtest,3.0,5.0.0,en,2023-07-31T10:33:44.953,true
1596,ner_posology_langtest,3.0,5.0.0,en,2023-07-28T13:50:27.340,true


In [0]:
# find oncology NER models

models = compatibility.findVersion("ner_oncology")

models_df = pd.DataFrame([dict(x) for x in list(models)])
models_df

Out[4]:

,name,sparkVersion,version,language,date,readyToUse
0,ner_oncology_wip,3.0,3.5.0,en,2022-07-25T17:48:26.053,true
1,ner_oncology_unspecific_posology_wip,3.0,4.0.0,en,2022-09-30T18:49:12.336,true
2,ner_oncology_wip,3.0,4.0.0,en,2022-09-30T16:54:45.893,true
3,ner_oncology_therapy_wip,3.0,4.0.0,en,2022-09-30T17:12:16.894,true
4,ner_oncology_posology_wip,3.0,4.0.0,en,2022-10-01T04:46:44.423,true
...,...,...,...,...,...,...
105,ner_oncology_test_pipeline,3.0,4.4.4,en,2023-06-17T09:44:19.850,true
106,ner_oncology_diagnosis_pipeline,3.0,4.4.4,en,2023-06-17T09:39:59.043,true
107,ner_oncology_unspecific_posology_pipeline,3.0,4.4.4,en,2023-06-17T09:50:57.012,true
108,ner_oncology_demographics_pipeline,3.0,4.4.4,en,2023-06-17T09:37:46.228,true


Clinical NER Pipeline (with pretrained models)

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
 
# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")\
    .setLabelCasing("upper") #decide if we want to return the tags in upper or lower case 

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter
        ])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
ner_clinical_large download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
#checking the stages in pipeline
model.stages

Out[6]: [DocumentAssembler_763b3101e3c6,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_807272bc79f3,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NER_CONVERTER_d7bcf76f1443]

In [0]:
#getting the classes in pretrained model
clinical_ner.getClasses()

Out[7]: ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST']

In [0]:
#extracting the embedded default param values
clinical_ner.extractParamMap()

Out[8]: {Param(parent='MedicalNerModel_1a8637089929', name='includeAllConfidenceScores', doc='whether to include all confidence scores in annotation metadata or just the score of the predicted tag'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='inferenceBatchSize', doc='number of sentences to process in a single batch during inference'): 1,
 Param(parent='MedicalNerModel_1a8637089929', name='labelCasing', doc='Setting all labels of the NER models upper/lower case. values upper|lower'): 'upper',
 Param(parent='MedicalNerModel_1a8637089929', name='sentenceTokenIndex', doc='Whether to include the token index for each sentence in annotation metadata.'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='includeConfidence', doc='whether to include confidence scores in annotation metadata'): True,
 Param(parent='MedicalNerModel_1a8637089929', name='batchSize', doc='Size of every batch'): 64,
 Param(parent='MedicalNerModel_1a8637089929', name='classes', doc='get the tags used to trained this MedicalNerModel'): ['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST'],
 Param(parent='MedicalNerModel_1a8637089929', name='inputCols', doc='previous annotations columns, if renamed'): ['sentence',
 'token',
 'embeddings'],
 Param(parent='MedicalNerModel_1a8637089929', name='outputCol', doc='output annotation column. can be left default.'): 'ner',
 Param(parent='MedicalNerModel_1a8637089929', name='storageRef', doc='unique reference name for identification'): 'clinical'}

In [0]:
#checking the embeddings
clinical_ner.getStorageRef()

Out[9]: 'clinical'

In [0]:
#downloading the sample dataset
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mt_samples_10.csv

dbutils.fs.cp("file:/databricks/driver/mt_samples_10.csv", "dbfs:/") 

Out[10]: True

In [0]:
mt_samples_df = spark.createDataFrame(pd.read_csv("mt_samples_10.csv", sep=',', index_col=["index"]).reset_index())

In [0]:
mt_samples_df.printSchema()

root
-- index: long (nullable = true)
-- text: string (nullable = true)

In [0]:
mt_samples_df.show()

+-----+--------------------+
index| text|
+-----+--------------------+
 0|Sample Type / Med...|
 1|Sample Type / Med...|
 2|Sample Type / Med...|
 3|Sample Type / Med...|
 4|Sample Type / Med...|
 5|Sample Type / Med...|
 6|Sample Type / Med...|
 7|Sample Type / Med...|
 8|Sample Type / Med...|
 9|Sample Type / Med...|
+-----+--------------------+

In [0]:
print(mt_samples_df.limit(1).collect()[0]['text'])

Sample Type / Medical Specialty:
Hematology - Oncology
Sample Name:
Discharge Summary - Mesothelioma - 1
Description:
Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
(Medical Transcription Sample Report)
PRINCIPAL DIAGNOSIS:
Mesothelioma.
SECONDARY DIAGNOSES:
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
PROCEDURES
1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.
2. On August 20, 2007, thoracentesis.
3. On August 31, 2007, Port-A-Cath placement.
HISTORY AND PHYSICAL:
The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.
PAST MEDICAL HISTORY
1. Pericardectomy.
2. Pericarditis.
2. Atrial fibrillation.
4. RNCA with intracranial thrombolytic treatment.
5 PTA of MCA.
6. Mesenteric venous thrombosis.
7. Pericardial window.
8. Cholecystectomy.
9. Left thoracentesis.
FAMILY HISTORY:
No family history of coronary artery disease, CVA, diabetes, CHF or MI. The patient has one family member, a sister, with history of cancer.
SOCIAL HISTORY:
She is married. Employed with the US Post Office. She is a mother of three. Denies tobacco, alcohol or illicit drug use.
MEDICATIONS
1. Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and her INR was 2.3.
2. Amiodarone 100 mg p.o. daily.
REVIEW OF SYSTEMS:
Complete review of systems negative except as in pulmonary as noted above. The patient also reports occasional numbness and tingling of her left arm.
PHYSICAL EXAMINATION
VITAL SIGNS: Blood pressure 123/95, heart rate 83, respirations 20, temperature 97, and oxygen saturation 97%.
GENERAL: Positive nonproductive cough and pain with coughing.
HEENT: Pupils are equal and reactive to light and accommodation. Tympanic membranes are clear.
NECK: Supple. No lymphadenopathy. No masses.
RESPIRATORY: Pleural friction rub is noted.
GI: Soft, nondistended, and nontender. Positive bowel sounds. No organomegaly.
EXTREMITIES: No edema, no clubbing, no cyanosis, no tenderness. Full range of motion. Normal pulses in all extremities.
SKIN: No breakdown or lesions. No ulcers.
NEUROLOGIC: Grossly intact. No focal deficits. Awake, alert, and oriented to person, place, and time.
LABORATORY DATA:
Labs are pending.
HOSPITAL COURSE:
The patient was admitted for a right-sided pleural effusion for thoracentesis on Monday by Dr. X. Her Coumadin was placed on hold. A repeat echocardiogram was checked. She was started on prophylaxis for DVT with Lovenox 40 mg subcutaneously. Her history dated back to March 2005 when she first sought medical attention for evidence of pericarditis, which was treated with pericardial window in an outside hospital, at that time she was also found to have mesenteric pain and thrombosis, is now anticoagulated. Her pericardial fluid was accumulated and she was seen by Dr. Y. At that time, she was recommended for pericardectomy, which was performed by Dr. Z. Review of her CT scan from March 2006 prior to her pericardectomy, already shows bilateral plural effusions. The patient improved clinically after the pericardectomy with resolution of her symptoms. Recently, she was readmitted to the hospital with chest pain and found to have bilateral pleural effusion, the right greater than the left. CT of the chest also revealed a large mediastinal lymph node. We reviewed the pathology obtained from the pericardectomy in March 2006, which was diagnostic of mesothelioma. At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid mal

In [0]:
result = model.transform(mt_samples_df).cache()

In [0]:
result.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
index| text| document| sentence| token| embeddings| ner| ner_chunk|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 0|Sample Type / Med...|[{document, 0, 54...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 118, 129...|
 1|Sample Type / Med...|[{document, 0, 32...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 92, ...|
 2|Sample Type / Med...|[{document, 0, 42...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 73, ...|
 3|Sample Type / Med...|[{document, 0, 20...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 119, 155...|
 4|Sample Type / Med...|[{document, 0, 34...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 97, 116,...|
 5|Sample Type / Med...|[{document, 0, 15...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 84, ...|
 6|Sample Type / Med...|[{document, 0, 25...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 73, ...|
 7|Sample Type / Med...|[{document, 0, 93...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 92, ...|
 8|Sample Type / Med...|[{document, 0, 20...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 155, 189...|
 9|Sample Type / Med...|[{document, 0, 19...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 68, 95, ...|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
%%time

result.select('token.result','ner.result').show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Discharge, Sum...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, O, B-PROBLEM, I-PROBLEM, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, BCCa, Excision...|[O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATM...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Anemia, -, Con...|[O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, O, O, O, O, B-PROBLEM, I-PROBLEM, O, O, B-PROBLEM...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Intensity-Modu...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, I-TREATMENT, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Neck, Dissecti...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PROBLEM, I...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, HDR, Brachythe...|[O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, B-TREATMENT, I-TREATMENT, O,...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Biopsy, -, Cer...|[O, O, O, O, O, O, O, O, O, O, O, O, B-TEST, O, O, O, O, O, O, B-TEST, I-TEST, I-TEST, I-TEST, I-...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Metastatic, Ov...|[O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Breast, Radiat...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Lymph, Node, E...|[O, O, O, O, O, O, O, O, O, O, O, O, B-TEST, I-TEST, I-TEST, I-TEST, O, O, B-TEST, I-TEST, I-TEST...|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+

CPU times: user 0 ns, sys: 2.19 ms, total: 2.19 ms
Wall time: 479 ms

In [0]:
result_df = result.select(F.explode(F.arrays_zip(result.token.result,
                                                 result.ner.result, 
                                                 result.ner.metadata)).alias("cols"))\
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"),
                          F.expr("cols['2']['confidence']").alias("confidence"))

result_df.show(50, truncate=100)

+-------------+---------+----------+
 token|ner_label|confidence|
+-------------+---------+----------+
 Sample| O| 0.9996|
 Type| O| 0.9926|
 /| O| 0.9755|
 Medical| O| 0.9824|
 Specialty| O| 0.9993|
 :| O| 1.0|
 Hematology| O| 0.9971|
 -| O| 0.9908|
 Oncology| O| 0.9796|
 Sample| O| 0.9999|
 Name| O| 0.9994|
 :| O| 1.0|
 Discharge| O| 0.9998|
 Summary| O| 0.9926|
 -| O| 0.9959|
 Mesothelioma| O| 0.8117|
 -| O| 0.9934|
 1| O| 0.7394|
 Description| O| 0.9995|
 :| O| 0.9999|
 Mesothelioma|B-PROBLEM| 0.9991|
 ,| O| 0.9999|
 pleural|B-PROBLEM| 0.9911|
 effusion|I-PROBLEM| 0.9982|
 ,| O| 0.9999|
 atrial|B-PROBLEM| 0.9976|
 fibrillation|I-PROBLEM| 0.9982|
 ,| O| 0.9999|
 anemia|B-PROBLEM| 0.999|
 ,| O| 0.9999|
 ascites|B-PROBLEM| 0.9996|
 ,| O| 1.0|
 esophageal|B-PROBLEM| 0.9982|
 reflux|I-PROBLEM| 0.9976|
 ,| O| 1.0|
 and| O| 0.9996|
 history| O| 0.9892|
 of| O| 0.9769|
 deep|B-PROBLEM| 0.9421|
 venous|I-PROBLEM| 0.9624|
 thrombosis|I-PROBLEM| 0.9983|
 .| O| 1.0|
 (| O| 0.9999|
 Medical| O| 0.986|
Transcription| O| 0.9214|
 Sample| O| 0.993|
 Report| O| 0.9536|
 )| O| 0.9802|
 PRINCIPAL| O| 0.9981|
 DIAGNOSIS| O| 0.4728|
+-------------+---------+----------+
only showing top 50 rows

In [0]:
result_df.select("token", "ner_label")\
         .groupBy('ner_label').count()\
         .orderBy('count', ascending=False)\
         .show(truncate=False)

+-----------+-----+
ner_label |count|
+-----------+-----+
O |4721 |
I-PROBLEM |526 |
B-PROBLEM |399 |
I-TREATMENT|326 |
B-TREATMENT|265 |
I-TEST |173 |
B-TEST |142 |
+-----------+-----+

In [0]:
result.select('ner_chunk').take(1)

Out[20]: [Row(ner_chunk=[Row(annotatorType='chunk', begin=118, end=129, result='Mesothelioma', metadata={'sentence': '2', 'chunk': '0', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.9991'}, embeddings=[]), Row(annotatorType='chunk', begin=132, end=147, result='pleural effusion', metadata={'sentence': '2', 'chunk': '1', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.99465'}, embeddings=[]), Row(annotatorType='chunk', begin=150, end=168, result='atrial fibrillation', metadata={'sentence': '2', 'chunk': '2', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.9979'}, embeddings=[]), Row(annotatorType='chunk', begin=171, end=176, result='anemia', metadata={'sentence': '2', 'chunk': '3', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.999'}, embeddings=[]), Row(annotatorType='chunk', begin=179, end=185, result='ascites', metadata={'sentence': '2', 'chunk': '4', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.9996'}, embeddings=[]), Row(annotatorType='chunk', begin=188, end=204, result='esophageal reflux', metadata={'sentence': '2', 'chunk': '5', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.9979'}, embeddings=[]), Row(annotatorType='chunk', begin=222, end=243, result='deep venous thrombosis', metadata={'sentence': '2', 'chunk': '6', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.96760005'}, embeddings=[]), Row(annotatorType='chunk', begin=305, end=316, result='Mesothelioma', metadata={'sentence': '4', 'chunk': '7', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.987'}, embeddings=[]), Row(annotatorType='chunk', begin=340, end=355, result='Pleural effusion', metadata={'sentence': '5', 'chunk': '8', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.97895'}, embeddings=[]), Row(annotatorType='chunk', begin=358, end=376, result='atrial fibrillation', metadata={'sentence': '5', 'chunk': '9', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.99475'}, embeddings=[]), Row(annotatorType='chunk', begin=379, end=384, result='anemia', metadata={'sentence': '5', 'chunk': '10', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.9984'}, embeddings=[]), Row(annotatorType='chunk', begin=387, end=393, result='ascites', metadata={'sentence': '5', 'chunk': '11', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.9994'}, embeddings=[]), Row(annotatorType='chunk', begin=396, end=412, result='esophageal reflux', metadata={'sentence': '5', 'chunk': '12', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.99724996'}, embeddings=[]), Row(annotatorType='chunk', begin=430, end=451, result='deep venous thrombosis', metadata={'sentence': '5', 'chunk': '13', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'confidence': '0.9550001'}, embeddings=[]), Row(annotatorType='chunk', begin=488, end=512, result='decortication of the lung', metadata={'sentence': '8', 'chunk': '14', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'confidence': '0.756325'}, embeddings=[]), Row(annotatorType='chunk', begin=519, end=532, result='pleural biopsy', metadata={'sentence': '8', 'chunk': '15', 'ner_source': 'ner_chunk', 'entity': 'TEST', 'confidence': '0.87715'}, embeddings=[]), Row(annotatorType='chunk', begin=538, end=561, result='transpleural fluoroscopy', metadata={'sentence': '8', 'chunk': '16', 'ner_source': 'ner_chunk', 'entity': 'TEST', 'confidence': '0.9527'}, embeddings=[]), Row(annotatorType='chunk', begin=587, end=599, result='thoracentesis', metadata={'sentence': '9', 'chunk': '17', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'confidence': '0.9075'}, embeddings=[]), Row(annotatorType='chunk', begin=625, end=645, result='Port-A-Cath placement', metadata={'sentence': '10', 'chunk': '18', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'confidence': '0.98075'}, embeddings=[]), Row(annotatorType='chunk', begin=722, end=742, result='a nonproductive cough', met

In [0]:
row_list = [{'annotatorType': row.annotatorType, 
             'begin': row.begin, 
             'end': row.end,
             'result': row.result,
             'metadata': row.metadata} 
            for row in result.select('ner_chunk').take(1)[0][0]
           ]

row_list

Out[21]: [{'annotatorType': 'chunk',
 'begin': 118,
 'end': 129,
 'result': 'Mesothelioma',
 'metadata': {'sentence': '2',
 'chunk': '0',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.9991'}},
 {'annotatorType': 'chunk',
 'begin': 132,
 'end': 147,
 'result': 'pleural effusion',
 'metadata': {'sentence': '2',
 'chunk': '1',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.99465'}},
 {'annotatorType': 'chunk',
 'begin': 150,
 'end': 168,
 'result': 'atrial fibrillation',
 'metadata': {'sentence': '2',
 'chunk': '2',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.9979'}},
 {'annotatorType': 'chunk',
 'begin': 171,
 'end': 176,
 'result': 'anemia',
 'metadata': {'sentence': '2',
 'chunk': '3',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.999'}},
 {'annotatorType': 'chunk',
 'begin': 179,
 'end': 185,
 'result': 'ascites',
 'metadata': {'sentence': '2',
 'chunk': '4',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.9996'}},
 {'annotatorType': 'chunk',
 'begin': 188,
 'end': 204,
 'result': 'esophageal reflux',
 'metadata': {'sentence': '2',
 'chunk': '5',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.9979'}},
 {'annotatorType': 'chunk',
 'begin': 222,
 'end': 243,
 'result': 'deep venous thrombosis',
 'metadata': {'sentence': '2',
 'chunk': '6',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.96760005'}},
 {'annotatorType': 'chunk',
 'begin': 305,
 'end': 316,
 'result': 'Mesothelioma',
 'metadata': {'sentence': '4',
 'chunk': '7',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.987'}},
 {'annotatorType': 'chunk',
 'begin': 340,
 'end': 355,
 'result': 'Pleural effusion',
 'metadata': {'sentence': '5',
 'chunk': '8',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.97895'}},
 {'annotatorType': 'chunk',
 'begin': 358,
 'end': 376,
 'result': 'atrial fibrillation',
 'metadata': {'sentence': '5',
 'chunk': '9',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.99475'}},
 {'annotatorType': 'chunk',
 'begin': 379,
 'end': 384,
 'result': 'anemia',
 'metadata': {'sentence': '5',
 'chunk': '10',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.9984'}},
 {'annotatorType': 'chunk',
 'begin': 387,
 'end': 393,
 'result': 'ascites',
 'metadata': {'sentence': '5',
 'chunk': '11',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.9994'}},
 {'annotatorType': 'chunk',
 'begin': 396,
 'end': 412,
 'result': 'esophageal reflux',
 'metadata': {'sentence': '5',
 'chunk': '12',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.99724996'}},
 {'annotatorType': 'chunk',
 'begin': 430,
 'end': 451,
 'result': 'deep venous thrombosis',
 'metadata': {'sentence': '5',
 'chunk': '13',
 'ner_source': 'ner_chunk',
 'entity': 'PROBLEM',
 'confidence': '0.9550001'}},
 {'annotatorType': 'chunk',
 'begin': 488,
 'end': 512,
 'result': 'decortication of the lung',
 'metadata': {'sentence': '8',
 'chunk': '14',
 'ner_source': 'ner_chunk',
 'entity': 'TREATMENT',
 'confidence': '0.756325'}},
 {'annotatorType': 'chunk',
 'begin': 519,
 'end': 532,
 'result': 'pleural biopsy',
 'metadata': {'sentence': '8',
 'chunk': '15',
 'ner_source': 'ner_chunk',
 'entity': 'TEST',
 'confidence': '0.87715'}},
 {'annotatorType': 'chunk',
 'begin': 538,
 'end': 561,
 'result': 'transpleural fluoroscopy',
 'metadata': {'sentence': '8',
 'chunk': '16',
 'ner_source': 'ner_chunk',
 'entity': 'TEST',
 'confidence': '0.9527'}},
 {'annotatorType': 'chunk',
 'begin': 587,
 'end': 599,
 'result': 'thoracentesis',
 'metadata': {'sentence': '9',
 'chunk': '17',
 'ner_source': 'ner_chunk',
 'entity': 'TREATMENT',
 'confidence': '0.9075'}},
 {'annotatorType': 'chunk',
 'begin': 625,
 'end': 645,
 'result': 'Port-A-Cath placement',
 'metadata': {'sentence': '10',
 'chunk': '18',
 'ner_source': 'ner_chunk',
 'entity': 'TREATMENT',
 'confidence

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['1']['confidence']").alias("confidence")).show(truncate=False)

+-------------------------+---------+----------+
chunk |ner_label|confidence|
+-------------------------+---------+----------+
Mesothelioma |PROBLEM |0.9991 |
pleural effusion |PROBLEM |0.99465 |
atrial fibrillation |PROBLEM |0.9979 |
anemia |PROBLEM |0.999 |
ascites |PROBLEM |0.9996 |
esophageal reflux |PROBLEM |0.9979 |
deep venous thrombosis |PROBLEM |0.96760005|
Mesothelioma |PROBLEM |0.987 |
Pleural effusion |PROBLEM |0.97895 |
atrial fibrillation |PROBLEM |0.99475 |
anemia |PROBLEM |0.9984 |
ascites |PROBLEM |0.9994 |
esophageal reflux |PROBLEM |0.99724996|
deep venous thrombosis |PROBLEM |0.9550001 |
decortication of the lung|TREATMENT|0.756325 |
pleural biopsy |TEST |0.87715 |
transpleural fluoroscopy |TEST |0.9527 |
thoracentesis |TREATMENT|0.9075 |
Port-A-Cath placement |TREATMENT|0.98075 |
a nonproductive cough |PROBLEM |0.96173334|
+-------------------------+---------+----------+
only showing top 20 rows

### with LightPipelines

In [0]:
# fullAnnotate in LightPipeline

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting. 
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. 
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27. 
'''

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    

df_clinical = pd.DataFrame({'chunks':chunks, 
                            'begin': begin, 
                            'end':end, 
                            'sentence_id':sentence, 
                            'entities':entities})

df_clinical.head(20)

A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting. 
She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. 
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27. 

Out[23]:

,chunks,begin,end,sentence_id,entities
0,gestational diabetes mellitus,40,68,0,PROBLEM
1,subsequent type two diabetes mellitus,118,154,0,PROBLEM
2,T2DM,157,160,0,PROBLEM
3,HTG-induced pancreatitis,185,208,0,PROBLEM
4,an acute hepatitis,265,282,0,PROBLEM
5,polyuria,322,329,0,PROBLEM
6,poor appetite,332,344,0,PROBLEM
7,vomiting,351,358,0,PROBLEM
8,metformin,373,381,1,TREATMENT
9,glipizide,384,392,1,TREATMENT


In [0]:
light_result[0]

Out[24]: {'document': [Annotation(document, 0, 881, 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting. 
 She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. 
 Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27. 
 , {}, [])],
 'ner_chunk': [Annotation(chunk, 40, 68, gestational diabetes mellitus, {'chunk': '0', 'confidence': '0.91976666', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 118, 154, subsequent type two diabetes mellitus, {'chunk': '1', 'confidence': '0.75924003', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 157, 160, T2DM, {'chunk': '2', 'confidence': '0.9917', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 185, 208, HTG-induced pancreatitis, {'chunk': '3', 'confidence': '0.97535', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 265, 282, an acute hepatitis, {'chunk': '4', 'confidence': '0.9440667', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 322, 329, polyuria, {'chunk': '5', 'confidence': '0.9728', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 332, 344, poor appetite, {'chunk': '6', 'confidence': '0.9934', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 351, 358, vomiting, {'chunk': '7', 'confidence': '0.9854', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '0'}, []),
 Annotation(chunk, 373, 381, metformin, {'chunk': '8', 'confidence': '0.9998', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'sentence': '1'}, []),
 Annotation(chunk, 384, 392, glipizide, {'chunk': '9', 'confidence': '0.9999', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'sentence': '1'}, []),
 Annotation(chunk, 399, 411, dapagliflozin, {'chunk': '10', 'confidence': '1.0', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'sentence': '1'}, []),
 Annotation(chunk, 417, 420, T2DM, {'chunk': '11', 'confidence': '0.9995', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '1'}, []),
 Annotation(chunk, 426, 437, atorvastatin, {'chunk': '12', 'confidence': '0.9999', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'sentence': '1'}, []),
 Annotation(chunk, 443, 453, gemfibrozil, {'chunk': '13', 'confidence': '1.0', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'sentence': '1'}, []),
 Annotation(chunk, 459, 461, HTG, {'chunk': '14', 'confidence': '0.9994', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '1'}, []),
 Annotation(chunk, 480, 492, dapagliflozin, {'chunk': '15', 'confidence': '0.9999', 'ner_source': 'ner_chunk', 'entity': 'TREATMENT', 'sentence': '2'}, []),
 Annotation(chunk, 539, 558, Physical examination, {'chunk': '16', 'confidence': '0.9448', 'ner_source': 'ner_chunk', 'entity': 'TEST', 'sentence': '3'}, []),
 Annotation(chunk, 596, 610, dry oral mucosa, {'chunk': '17', 'confidence': '0.7242333', 'ner_source': 'ner_chunk', 'entity': 'PROBLEM', 'sentence': '3'}, []),
 Annotation(chunk, 630, 654, her abdominal examination, {'chunk': '18', 'confidence': '0.8749666', 'ner_source': 'ner_chunk', 'entity': 'TEST', 'sentence': '3'}, []),
 Annotation(chunk, 675, 684, tenderness, {'chunk': '19', '

**Getting embeddings with LP**

In [0]:
light_model_with_emb = LightPipeline(model, parse_embeddings=True)
light_result_with_emb = light_model_with_emb.annotate(text)

list(zip(light_result_with_emb['token'], light_result_with_emb['embeddings']))[:3]

Out[25]: [('A',
 '0.040217556 0.4003962 0.69088835 -0.44649735 -0.40282863 -0.45621523 -0.20415598 0.13707675 -0.010474405 -0.42899358 -0.2879874 -0.03076839 -0.40367207 -0.32588458 0.23294163 -0.16012932 -0.35798857 0.043061644 -0.4091242 0.101141304 -0.61503464 0.068771854 -0.025053142 0.11466996 -0.57737786 -0.25603554 -0.6950373 0.3328502 -0.100428 0.0191588 -0.61035156 -0.94459176 -0.2473863 0.16416614 -0.27778018 0.40670344 0.57310873 0.10776807 0.13333033 0.1447822 0.18491237 0.2196365 0.16771133 -0.40313882 -0.20199609 0.045175955 -0.2593916 -0.066459864 0.15229034 0.002910823 -0.16859493 0.17066914 -9.03164E-4 0.03531517 -0.33315992 -0.034506235 0.40311962 0.023356766 0.13823766 -0.2634582 0.27318013 -0.15994109 0.40696687 0.19050935 -0.19682823 -0.012536205 0.4694685 -0.14188287 -0.024362855 -0.38291803 0.013956237 0.1830807 0.34642568 -0.23263279 -0.14591216 -0.2337688 0.5834314 -0.03057853 -0.367189 0.56245005 -0.032807935 0.122141086 0.17941982 -0.00749746 -0.18216385 0.17958473 -0.3053265 0.22779782 0.02217061 0.029920649 0.17621519 0.011366806 0.0385044 0.10249041 0.3426693 -0.035018172 0.4671417 0.22892669 -0.30640814 0.10803514 0.3855227 -0.115776226 -0.44415653 -0.11332124 0.044015724 0.61063313 0.29600143 -0.012069431 -0.38224095 -0.13973162 0.39301684 0.23673905 0.057245154 0.18140107 -0.051603805 0.7974051 -0.06663643 -0.5508283 0.28209952 0.18742235 0.21872385 -0.25591275 0.066957995 -0.25298932 0.13821244 0.8307903 0.3973212 -0.17276311 0.12846412 -0.092758246 0.0037952603 -0.41248038 0.65204984 -0.03299473 0.21624717 -0.56830674 0.23451968 -0.05922969 -0.3901237 -0.40775988 0.10331385 -0.0052452036 0.17687175 -0.2470756 0.054437738 0.10611684 -0.546031 0.03243354 -0.10689023 -0.39552742 -0.29755193 -0.12058273 0.047433637 0.16503441 -0.033334065 0.015473544 0.026549816 -0.7305153 -0.09848879 0.11045749 0.41713542 -0.13625173 -0.4153576 0.23202501 0.45888668 -0.0958802 -0.1448712 -0.63727266 0.26895493 -0.2672995 0.08526398 -0.14055037 8.3983614E-4 0.23264983 -0.13427801 0.4620594 0.33161256 0.43790993 0.1952479 -0.27517807 0.22294433 -0.28702205 0.36680275 -0.2888683 -0.10005434 0.029136676 -0.19198024 -0.05411223 0.19856672 0.041492134 0.042284537 0.27730006 0.33647385 -0.25736117 0.10052982 0.0914106 -0.5406019 -0.18598284 -0.54862744 -0.43890363'),
 ('28-year-old',
 '0.124630086 0.041768234 -0.5851833 0.14018573 -0.2683909 -0.6359739 -0.31614816 -0.71483403 0.5511774 0.4304843 -0.5115899 0.7928865 0.0788141 0.0055778096 -0.10271598 -0.5141132 -0.22763066 0.25965604 0.059446163 0.32050937 -0.16531321 -0.0833686 0.82620156 0.22395885 -0.1965371 -0.11455178 -0.5434175 0.2595536 -0.0153490305 -0.474383 0.14677128 -0.080003664 -0.052671567 -0.04031274 -0.22524078 -0.35215047 0.046719514 -0.21722157 -0.32051668 0.23190966 -0.04614518 1.0149231 0.6315361 -0.47952643 0.44472834 0.053033605 0.31127706 0.08055067 -0.7282213 -0.7802679 0.16257171 0.073258646 0.40050963 0.44751948 0.4896496 -0.17256223 0.1598703 -0.65302587 -0.06466415 -0.21638417 0.3140914 0.4975012 -0.12023235 0.09910327 -0.04403189 -0.8829349 0.6587716 0.17046705 0.1637326 0.28226846 0.19858298 0.37132013 0.6070592 -0.11720922 -0.1534436 0.15268378 -0.055180248 -0.23013055 -0.46350223 0.25226158 -0.4663529 0.19255444 -0.5118875 0.1125828 -0.3138261 0.11908515 -0.17581931 -0.48491156 0.20515475 -0.4273081 -0.6629976 0.31330186 0.3916261 -0.056986686 0.11155176 -0.030073252 0.1524556 -0.36797798 0.6462149 -0.08456914 0.050772805 -0.34014466 0.3223971 0.015675943 0.015383259 0.34324646 0.096488 -0.33835164 0.067233734 0.6365546 0.52120185 0.37197444 -0.027830597 0.4061612 0.028669652 0.27176672 0.07524431 -0.55174196 0.28766853 0.07589323 -0.22518751 -0.44933867 -0.4721748 -0.108820066 0.32532197 0.08391404 -0.45878386 0.26986977 0.1009148 0.6153148 -0.048685577 0.30017054 -0.40371877 0.047533885 0.53543746 -0.27002755 0.25372973 0.10320132 -0.14759818 -0.27956298 0.3565558 -0.1178739 0.75151694 0.39113513 0.17251517 0.20465879 0.11995484 0.8694

**Prettifying the results with NLU**

John Snow Labs' NLU is a Python library for applying state-of-the-art text mining, directly on any dataframe, with a single line of code. We will use some functionalities of NLU to prettify our results.

In [0]:
nlu.to_pretty_df(model, text, positions=True, output_level='chunk').columns

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[26]: Index(['document', 'document_begin', 'document_end', 'entities_ner_chunk',
 'entities_ner_chunk_begin', 'entities_ner_chunk_class',
 'entities_ner_chunk_confidence', 'entities_ner_chunk_end',
 'entities_ner_chunk_origin_chunk', 'entities_ner_chunk_origin_sentence',
 'ner_iob_ner_begin', 'ner_iob_ner_end', 'sentence_dl',
 'sentence_dl_begin', 'sentence_dl_end', 'token_begin', 'token_end',
 'word_embedding_embeddings', 'word_embedding_embeddings_begin',
 'word_embedding_embeddings_end'],
 dtype='object')

In [0]:
#For a given pipeline output level is automatically set to the last anntator's output level by default.
#This can be changed by defining the value of "output_level" parameter.
#Values can be set as; token, sentence, document, chunk and relation .

cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_begin',
     'entities_ner_chunk_end',
     'entities_ner_chunk_origin_sentence',
     'entities_ner_chunk_class',
]
df_clinical = nlu.to_pretty_df(model, text, positions=True, output_level='chunk')[cols]
df_clinical.head(20)

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[27]:

,entities_ner_chunk,entities_ner_chunk_begin,entities_ner_chunk_end,entities_ner_chunk_origin_sentence,entities_ner_chunk_class
0,gestational diabetes mellitus,40,68,0,PROBLEM
0,subsequent type two diabetes mellitus,118,154,0,PROBLEM
0,T2DM,157,160,0,PROBLEM
0,HTG-induced pancreatitis,185,208,0,PROBLEM
0,an acute hepatitis,265,282,0,PROBLEM
0,polyuria,322,329,0,PROBLEM
0,poor appetite,332,344,0,PROBLEM
0,vomiting,351,358,0,PROBLEM
0,metformin,373,381,1,TREATMENT
0,glipizide,384,392,1,TREATMENT


### NER Visualizer

For saving the visualization result as html, provide `save_path` parameter in the display function.

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

# Change color of an entity label

#visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
#visualiser.display(light_result[0], label_col='ner_chunk')

# Set label filter

# visualiser.display(light_result, label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST'])
  
displayHTML(ner_vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM ( T2DM PROBLEM ), one prior episode of HTG-induced pancreatitis PROBLEM three years prior to presentation, and associated with an acute hepatitis PROBLEM , presented with a one-week history of polyuria PROBLEM , poor appetite PROBLEM , and vomiting PROBLEM . She was on metformin TREATMENT , glipizide TREATMENT , and dapagliflozin TREATMENT for T2DM PROBLEM and atorvastatin TREATMENT and gemfibrozil TREATMENT for HTG PROBLEM . She had been on dapagliflozin TREATMENT for six months at the time of presentation. Physical examination TEST on presentation was significant for dry oral mucosa PROBLEM ; significantly , her abdominal examination TEST was benign with no tenderness PROBLEM , guarding PROBLEM , or rigidity PROBLEM . Pertinent laboratory findings on admission were: serum glucose TEST 111 mg/dl, creatinine TEST 0.4 mg/dL, triglycerides TEST 508 mg/dL, total cholesterol TEST 122 mg/dL, and venous pH TEST 7.27.

### ModelTracer

`ModelTracer` allows to track the uids and timestamps of each stage of the pipeline.

Let's apply it over an empty data with our existing pipeline.

In [0]:
from sparknlp_jsl.modelTracer import ModelTracer

data = spark.createDataFrame([['']]).toDF("text")

df = nlpPipeline.fit(data).transform(data)

tracer_result = ModelTracer().addUidCols(pipeline = nlpPipeline, df = df)

tracer_result.show(truncate=False)

+----+------------------------------------------+--------+-----+----------+---+---------+----------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------------------------------------------+--------------------------------------------------------------------------+--------------------------------------------------------------------+-------------------------------------------------------------------------+
text|document |sentence|token|embeddings|ner|ner_chunk|documentassembler_model_uid |sentencedetectordlmodel_model_uid |tokenizer_model_uid |word_embeddings_model_model_uid |medicalnermodel_model_uid |nerconverterinternal_model_uid |
+----+------------------------------------------+--------+-----+----------+---+---------+----------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------------------------------------------+--------------------------------------------------------------------------+--------------------------------------------------------------------+-------------------------------------------------------------------------+
 |[{document, 0, -1, , {sentence -> 0}, []}]|[] |[] |[] |[] |[] |{uid -> DocumentAssembler_763b3101e3c6, timestamp -> 2023-08-03_11:21}|{uid -> SentenceDetectorDLModel_6bafc4746ea5, timestamp -> 2023-08-03_11:21}|{uid -> Tokenizer_487dcd3b4af4, timestamp -> 2023-08-03_11:21}|{uid -> WORD_EMBEDDINGS_MODEL_9004b1d00302, timestamp -> 2023-08-03_11:21}|{uid -> MedicalNerModel_1a8637089929, timestamp -> 2023-08-03_11:21}|{uid -> NerConverterInternal_91f3132fc230, timestamp -> 2023-08-03_11:21}|
+----+------------------------------------------+--------+-----+----------+---+---------+----------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------------------------------------------+--------------------------------------------------------------------------+--------------------------------------------------------------------+-------------------------------------------------------------------------+

## NER JSL Model

Let's show an example of `ner_jsl` model that has about 80 labels by changing just only the model name.

**Entities**

| | | | | |
|-|-|-|-|-|
|Injury_or_Poisoning|Direction|Test|Admission_Discharge|Death_Entity|
|Relationship_Status|Duration|Respiration|Hyperlipidemia|Birth_Entity|
|Age|Labour_Delivery|Family_History_Header|BMI|Temperature|
|Alcohol|Kidney_Disease|Oncological|Medical_History_Header|Cerebrovascular_Disease|
|Oxygen_Therapy|O2_Saturation|Psychological_Condition|Heart_Disease|Employment|
|Obesity|Disease_Syndrome_Disorder|Pregnancy|ImagingFindings|Procedure|
|Medical_Device|Race_Ethnicity|Section_Header|Symptom|Treatment|
|Substance|Route|Drug_Ingredient|Blood_Pressure|Diet|
|External_body_part_or_region|LDL|VS_Finding|Allergen|EKG_Findings|
|Imaging_Technique|Triglycerides|RelativeTime|Gender|Pulse|
|Social_History_Header|Substance_Quantity|Diabetes|Modifier|Internal_organ_or_component|
|Clinical_Dept|Form|Drug_BrandName|Strength|Fetus_NewBorn|
|RelativeDate|Height|Test_Result|Sexually_Active_or_Sexual_Orientation|Frequency|
|Time|Weight|Vaccine|Vital_Signs_Header|Communicable_Disease|
|Dosage|Overweight|Hypertension|HDL|Total_Cholesterol|
|Smoking|Date||||

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") 

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("jsl_ner")
    
jsl_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "jsl_ner"]) \
    .setOutputCol("jsl_ner_chunk")

jsl_ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

jsl_ner_model = jsl_ner_pipeline.fit(empty_data)


sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl download started this may take some time.
[ | ][ / ][OK!]

In [0]:
jsl_light_model = LightPipeline(jsl_ner_model)

jsl_light_result = jsl_light_model.fullAnnotate(text)

In [0]:
cols = [
     'entities_jsl_ner_chunk',
     'entities_jsl_ner_chunk_begin',
     'entities_jsl_ner_chunk_end',
     'entities_jsl_ner_chunk_origin_sentence',
     'entities_jsl_ner_chunk_class',
]
jsl_df = nlu.to_pretty_df(jsl_ner_model, text,positions=True, output_level='chunk')[cols]
jsl_df.head(20)

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[32]:

,entities_jsl_ner_chunk,entities_jsl_ner_chunk_begin,entities_jsl_ner_chunk_end,entities_jsl_ner_chunk_origin_sentence,entities_jsl_ner_chunk_class
0,28-year-old,3,13,0,Age
0,female,15,20,0,Gender
0,gestational diabetes mellitus,40,68,0,Diabetes
0,eight years prior,80,96,0,RelativeDate
0,type two diabetes mellitus,129,154,0,Diabetes
0,T2DM,157,160,0,Diabetes
0,HTG-induced pancreatitis,185,208,0,Disease_Syndrome_Disorder
0,three years prior,210,226,0,RelativeDate
0,acute,268,272,0,Modifier
0,hepatitis,274,282,0,Disease_Syndrome_Disorder


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(jsl_light_result[0], label_col='jsl_ner_chunk', document_col='document', return_html=True)

displayHTML(ner_vis)

A 28-year-old Age female Gender with a history of gestational diabetes mellitus Diabetes diagnosed eight years prior RelativeDate to presentation and subsequent type two diabetes mellitus Diabetes ( T2DM Diabetes ), one prior episode of HTG-induced pancreatitis Disease_Syndrome_Disorder three years prior RelativeDate to presentation, and associated with an acute Modifier hepatitis Disease_Syndrome_Disorder , presented with a one-week Duration history of polyuria Symptom , poor appetite Symptom , and vomiting Symptom . She Gender was on metformin Drug_Ingredient , glipizide Drug_Ingredient , and dapagliflozin Drug_Ingredient for T2DM Diabetes and atorvastatin Drug_Ingredient and gemfibrozil Drug_Ingredient for HTG Hyperlipidemia . She Gender had been on dapagliflozin Drug_Ingredient for six months Duration at the time of presentation. Physical examination on presentation was significant for dry oral mucosa Symptom ; significantly , her Gender abdominal External_body_part_or_region examination was benign with no tenderness Symptom , guarding Symptom , or rigidity Symptom . Pertinent laboratory findings on admission Admission_Discharge were: serum glucose Test 111 mg/dl Test_Result , creatinine Test 0.4 mg/dL Test_Result , triglycerides Triglycerides 508 mg/dL Test_Result , total cholesterol 122 mg/dL Total_Cholesterol , and venous pH Test 7.27 Test_Result .

## Posology NER

**Entities**

- DOSAGE
- DRUG
- DURATION
- FORM
- FREQUENCY
- ROUTE
- STRENGTH

In [0]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

posology_ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

# greedy model
posology_ner_greedy = MedicalNerModel.pretrained("ner_posology_greedy", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_greedy")

ner_converter_greedy = NerConverterInternal()\
    .setInputCols(["sentence","token","ner_greedy"])\
    .setOutputCol("ner_chunk_greedy")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    posology_ner_converter,
    posology_ner_greedy,
    ner_converter_greedy])

empty_data = spark.createDataFrame([[""]]).toDF("text")

posology_model = nlpPipeline.fit(empty_data)


ner_posology download started this may take some time.
[ | ][ / ][OK!]
ner_posology_greedy download started this may take some time.
[ | ][ / ][OK!]

In [0]:
posology_ner.getClasses()

Out[35]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
posology_result = posology_model.transform(mt_samples_df).cache()

In [0]:
posology_result.show(10)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
index| text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 0|Sample Type / Med...|[{document, 0, 54...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 1609, 16...|[{named_entity, 0...|[{chunk, 1609, 16...|
 1|Sample Type / Med...|[{document, 0, 32...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 742, 750...|[{named_entity, 0...|[{chunk, 742, 753...|
 2|Sample Type / Med...|[{document, 0, 42...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 230, 240...|[{named_entity, 0...|[{chunk, 369, 385...|
 3|Sample Type / Med...|[{document, 0, 20...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...| []|
 4|Sample Type / Med...|[{document, 0, 34...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 1102, 11...|[{named_entity, 0...|[{chunk, 1102, 11...|
 5|Sample Type / Med...|[{document, 0, 15...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...| []|[{named_entity, 0...|[{chunk, 68, 84, ...|
 6|Sample Type / Med...|[{document, 0, 25...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 1242, 12...|[{named_entity, 0...|[{chunk, 389, 398...|
 7|Sample Type / Med...|[{document, 0, 93...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 667, 677...|[{named_entity, 0...|[{chunk, 667, 677...|
 8|Sample Type / Med...|[{document, 0, 20...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 495, 505...|[{named_entity, 0...|[{chunk, 495, 513...|
 9|Sample Type / Med...|[{document, 0, 19...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 524, 526...|[{named_entity, 0...|[{chunk, 1234, 12...|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
posology_result = posology_result.withColumn("id", monotonically_increasing_id())

posology_result.show(3)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
index| text| document| sentence| token| embeddings| ner| ner_chunk| ner_greedy| ner_chunk_greedy| id|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
 0|Sample Type / Med...|[{document, 0, 54...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 1609, 16...|[{named_entity, 0...|[{chunk, 1609, 16...| 0|
 1|Sample Type / Med...|[{document, 0, 32...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 742, 750...|[{named_entity, 0...|[{chunk, 742, 753...| 8589934592|
 2|Sample Type / Med...|[{document, 0, 42...|[{document, 0, 53...|[{token, 0, 5, Sa...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 230, 240...|[{named_entity, 0...|[{chunk, 369, 385...|17179869184|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
only showing top 3 rows

In [0]:
posology_result.select('token.result','ner.result').show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Discharge, Sum...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, BCCa, Excision...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Anemia, -, Con...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Intensity-Modu...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Neck, Dissecti...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, HDR, Brachythe...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Biopsy, -, Cer...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Metastatic, Ov...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Breast, Radiat...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Name, :, Lymph, Node, E...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+

In [0]:
posology_result.select('token.result','ner.result').show(5, truncate=80)

+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
 result| result|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Nam...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Nam...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Nam...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Nam...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
[Sample, Type, /, Medical, Specialty, :, Hematology, -, Oncology, Sample, Nam...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
only showing top 5 rows

In [0]:
from pyspark.sql import functions as F

posology_result.select(F.explode(F.arrays_zip(posology_result.token.result, 
                                              posology_result.ner.result, 
                                              posology_result.ner.metadata)).alias("cols")) \
               .select(F.expr("cols['0']").alias("token"),
                       F.expr("cols['1']").alias("ner_label"),
                       F.expr("cols['2']['confidence']").alias("confidence"))\
               .filter("ner_label!='O'")\
               .show(20, truncate=100)


+--------------+-----------+----------+
 token| ner_label|confidence|
+--------------+-----------+----------+
 Coumadin| B-DRUG| 0.9989|
 1| B-STRENGTH| 0.8618|
 mg| I-STRENGTH| 0.5132|
 daily|B-FREQUENCY| 0.9999|
 Amiodarone| B-DRUG| 0.9993|
 100| B-STRENGTH| 0.8538|
 mg| I-STRENGTH| 0.5216|
 p.o| B-ROUTE| 0.9992|
 daily|B-FREQUENCY| 0.9928|
 Coumadin| B-DRUG| 0.9915|
 Lovenox| B-DRUG| 0.8314|
 40| B-STRENGTH| 0.5199|
 mg| I-STRENGTH| 0.5121|
subcutaneously| B-ROUTE| 0.9887|
 chemotherapy| B-DRUG| 0.9984|
 cisplatin| B-DRUG| 0.9257|
 75| B-STRENGTH| 0.9574|
 mg/centimeter| I-STRENGTH| 0.5491|
 109| B-STRENGTH| 0.5699|
 mg| I-STRENGTH| 0.6355|
+--------------+-----------+----------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip(posology_result.ner_chunk.result, 
                                                   posology_result.ner_chunk.begin, 
                                                   posology_result.ner_chunk.end, 
                                             posology_result.ner_chunk.metadata)).alias("cols")) \
              .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['3']['entity']").alias("ner_label"),
                            F.expr("cols['3']['confidence']").alias("confidence"))\
              .filter("ner_label!='O'")\
              .show(truncate=False)

+---+-----------+----------------+-----+----+---------+----------+
id |sentence_id|chunk |begin|end |ner_label|confidence|
+---+-----------+----------------+-----+----+---------+----------+
0 |33 |Coumadin |1609 |1616|DRUG |0.9989 |
0 |33 |1 mg |1618 |1621|STRENGTH |0.6875 |
0 |33 |daily |1623 |1627|FREQUENCY|0.9999 |
0 |34 |Amiodarone |1696 |1705|DRUG |0.9993 |
0 |34 |100 mg |1707 |1712|STRENGTH |0.68770003|
0 |34 |p.o |1714 |1716|ROUTE |0.9992 |
0 |34 |daily |1719 |1723|FREQUENCY|0.9928 |
0 |58 |Coumadin |2770 |2777|DRUG |0.9915 |
0 |60 |Lovenox |2880 |2886|DRUG |0.8314 |
0 |60 |40 mg |2888 |2892|STRENGTH |0.51600003|
0 |60 |subcutaneously |2894 |2907|ROUTE |0.9887 |
0 |72 |chemotherapy |4436 |4447|DRUG |0.9984 |
0 |72 |cisplatin |4475 |4483|DRUG |0.9257 |
0 |72 |75 mg/centimeter|4485 |4500|STRENGTH |0.75325 |
0 |72 |109 mg |4519 |4524|STRENGTH |0.6027 |
0 |72 |IV |4526 |4527|ROUTE |0.9993 |
0 |72 |piggyback |4529 |4537|DRUG |0.578 |
0 |72 |over 2 hours |4539 |4550|DURATION |0.7061 |
0 |72 |September |4555 |4563|DRUG |0.9277 |
0 |72 |Alimta |4574 |4579|DRUG |0.8845 |
+---+-----------+----------------+-----+----+---------+----------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip(posology_result.ner_chunk_greedy.result, 
                                                   posology_result.ner_chunk_greedy.begin, 
                                                   posology_result.ner_chunk_greedy.end, 
                                                   posology_result.ner_chunk_greedy.metadata)).alias("cols")) \
              .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
                            F.expr("cols['0']").alias("chunk"),
                            F.expr("cols['1']").alias("begin"),
                            F.expr("cols['2']").alias("end"),
                            F.expr("cols['3']['entity']").alias("ner_label"),
                            F.expr("cols['3']['confidence']").alias("confidence"))\
              .filter("ner_label!='O'")\
              .show(truncate=False)

+----------+-----------+--------------------------------------------------------------+-----+----+---------+----------+
id |sentence_id|chunk |begin|end |ner_label|confidence|
+----------+-----------+--------------------------------------------------------------+-----+----+---------+----------+
0 |33 |Coumadin 1 mg |1609 |1621|DRUG |0.6126 |
0 |33 |daily |1623 |1627|FREQUENCY|0.9962 |
0 |34 |Amiodarone 100 mg p.o |1696 |1716|DRUG |0.64115 |
0 |34 |daily |1719 |1723|FREQUENCY|0.9035 |
0 |58 |Coumadin |2770 |2777|DRUG |0.99 |
0 |72 |chemotherapy |4436 |4447|DRUG |0.9973 |
0 |72 |cisplatin 75 mg/centimeter |4475 |4500|DRUG |0.6081667 |
0 |72 |109 mg IV piggyback |4519 |4537|DRUG |0.57665 |
0 |72 |over 2 hours |4539 |4550|DURATION |0.5085333 |
0 |72 |Alimta 500 mg/ centimeter squared equaling 730 mg IV piggyback|4574 |4635|DRUG |0.58996 |
0 |74 |chemotherapy |4713 |4724|DRUG |0.9905 |
0 |74 |IV fluid and IV |4814 |4828|DRUG |0.7181 |
0 |75 |heparin |4863 |4869|DRUG |0.6508 |
0 |76 |Zofran |4917 |4922|DRUG |0.9919 |
0 |76 |Phenergan |4925 |4933|DRUG |0.9978 |
0 |76 |Coumadin |4936 |4943|DRUG |0.9989 |
0 |76 |Lovenox |4950 |4956|DRUG |0.9775 |
0 |76 |Vicodin |4963 |4969|DRUG |0.9946 |
8589934592|11 |Xylocaine 2% |742 |753 |DRUG |0.8614 |
8589934592|11 |epinephrine was injected |760 |783 |DRUG |0.8004667 |
+----------+-----------+--------------------------------------------------------------+-----+----+---------+----------+
only showing top 20 rows

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].result

Out[44]: 'Xylocaine'

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].metadata

Out[45]: {'sentence': '11',
 'chunk': '0',
 'ner_source': 'ner_chunk',
 'entity': 'DRUG',
 'confidence': '0.9906'}

In [0]:
posology_light_model = LightPipeline(posology_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'

posology_light_result = posology_light_model.annotate(text)

list(zip(posology_light_result['token'], posology_light_result['ner']))

Out[46]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
list(zip(posology_light_result['token'], posology_light_result['ner_greedy']))

Out[47]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DRUG'),
 ('capsule', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('Advil', 'I-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DRUG'),
 ('units', 'I-DRUG'),
 ('of', 'I-DRUG'),
 ('insulin', 'I-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'I-DRUG'),
 ('mg', 'I-DRUG'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('for', 'B-DURATION'),
 ('3', 'I-DURATION'),
 ('months', 'I-DURATION'),
 ('.', 'O')]

In [0]:
cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_class',
     'entities_ner_chunk_begin',
     'entities_ner_chunk_end',
]
posology_result_df = nlu.to_pretty_df(posology_light_model, text, positions=True, output_level='chunk')[cols]
posology_result_df.head(20)

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[48]:

,entities_ner_chunk,entities_ner_chunk_class,entities_ner_chunk_begin,entities_ner_chunk_end
0,1,DOSAGE,27,27
0,capsule,FORM,29,35
0,Advil,DRUG,40,44
0,for 5 days,DURATION,46,55
0,40 units,DOSAGE,126,133
0,insulin glargine,DRUG,138,153
0,at night,FREQUENCY,155,162
0,12 units,DOSAGE,166,173
0,insulin lispro,DRUG,178,191
0,with meals,FREQUENCY,193,202


In [0]:
cols = [
     'entities_ner_chunk_greedy',
     'entities_ner_chunk_greedy_class',
     'entities_ner_chunk_greedy_begin',
     'entities_ner_chunk_greedy_end',
]
posology_result_greedy_df = nlu.to_pretty_df(posology_light_model, text, positions=True, output_level='chunk')[cols]
posology_result_greedy_df.head(10)

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[49]:

,entities_ner_chunk_greedy,entities_ner_chunk_greedy_class,entities_ner_chunk_greedy_begin,entities_ner_chunk_greedy_end
0,1 capsule of Advil,DRUG,27,44
0,for 5 days,DURATION,46,55
0,40 units of insulin glargine,DRUG,126,153
0,at night,FREQUENCY,155,162
0,12 units of insulin lispro,DRUG,166,191
0,with meals,FREQUENCY,193,202
0,metformin 1000 mg,DRUG,210,226
0,two times a day,FREQUENCY,228,242
0,SGLT2 inhibitors,DRUG,273,288
0,for 3 months,DURATION,326,337


### NER Visualization

In [0]:
posology_light_result

Out[50]: {'document': ['The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .'],
 'ner_chunk': ['1',
 'capsule',
 'Advil',
 'for 5 days',
 '40 units',
 'insulin glargine',
 'at night',
 '12 units',
 'insulin lispro',
 'with meals',
 'metformin',
 '1000 mg',
 'two times a day',
 'SGLT2 inhibitors',
 'for 3 months'],
 'ner_greedy': ['O',
 'O',
 'O',
 'O',
 'B-DRUG',
 'I-DRUG',
 'I-DRUG',
 'I-DRUG',
 'B-DURATION',
 'I-DURATION',
 'I-DURATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DRUG',
 'I-DRUG',
 'I-DRUG',
 'I-DRUG',
 'I-DRUG',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'O',
 'B-DRUG',
 'I-DRUG',
 'I-DRUG',
 'I-DRUG',
 'I-DRUG',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'O',
 'O',
 'B-DRUG',
 'I-DRUG',
 'I-DRUG',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'I-FREQUENCY',
 'I-FREQUENCY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DRUG',
 'I-DRUG',
 'O',
 'O',
 'O',
 'O',
 'B-DURATION',
 'I-DURATION',
 'I-DURATION',
 'O'],
 'ner_chunk_greedy': ['1 capsule of Advil',
 'for 5 days',
 '40 units of insulin glargine',
 'at night',
 '12 units of insulin lispro',
 'with meals',
 'metformin 1000 mg',
 'two times a day',
 'SGLT2 inhibitors',
 'for 3 months'],
 'token': ['The',
 'patient',
 'was',
 'prescribed',
 '1',
 'capsule',
 'of',
 'Advil',
 'for',
 '5',
 'days',
 '.',
 'He',
 'was',
 'seen',
 'by',
 'the',
 'endocrinology',
 'service',
 'and',
 'she',
 'was',
 'discharged',
 'on',
 '40',
 'units',
 'of',
 'insulin',
 'glargine',
 'at',
 'night',
 ',',
 '12',
 'units',
 'of',
 'insulin',
 'lispro',
 'with',
 'meals',
 ',',
 'and',
 'metformin',
 '1000',
 'mg',
 'two',
 'times',
 'a',
 'day',
 '.',
 'It',
 'was',
 'determined',
 'that',
 'all',
 'SGLT2',
 'inhibitors',
 'should',
 'be',
 'discontinued',
 'indefinitely',
 'for',
 '3',
 'months',
 '.'],
 'ner': ['O',
 'O',
 'O',
 'O',
 'B-DOSAGE',
 'B-FORM',
 'O',
 'B-DRUG',
 'B-DURATION',
 'I-DURATION',
 'I-DURATION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DOSAGE',
 'I-DOSAGE',
 'O',
 'B-DRUG',
 'I-DRUG',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'O',
 'B-DOSAGE',
 'I-DOSAGE',
 'O',
 'B-DRUG',
 'I-DRUG',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'O',
 'O',
 'B-DRUG',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'I-FREQUENCY',
 'I-FREQUENCY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DRUG',
 'I-DRUG',
 'O',
 'O',
 'O',
 'O',
 'B-DURATION',
 'I-DURATION',
 'I-DURATION',
 'O'],
 'embeddings': ['The',
 'patient',
 'was',
 'prescribed',
 '1',
 'capsule',
 'of',
 'Advil',
 'for',
 '5',
 'days',
 '.',
 'He',
 'was',
 'seen',
 'by',
 'the',
 'endocrinology',
 'service',
 'and',
 'she',
 'was',
 'discharged',
 'on',
 '40',
 'units',
 'of',
 'insulin',
 'glargine',
 'at',
 'night',
 ',',
 '12',
 'units',
 'of',
 'insulin',
 'lispro',
 'with',
 'meals',
 ',',
 'and',
 'metformin',
 '1000',
 'mg',
 'two',
 'times',
 'a',
 'day',
 '.',
 'It',
 'was',
 'determined',
 'that',
 'all',
 'SGLT2',
 'inhibitors',
 'should',
 'be',
 'discontinued',
 'indefinitely',
 'for',
 '3',
 'months',
 '.'],
 'sentence': ['The patient was prescribed 1 capsule of Advil for 5 days .',
 'He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day .',
 'It was determined that all SGLT2 inhibitors should be discontinued indefinitely for 3 months .']}

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

posology_light_result = posology_light_model.fullAnnotate(text)

ner_vis = visualiser.display(posology_light_result[0], label_col='ner_chunk', document_col='document', return_html=True)
  
displayHTML(ner_vis)

The patient was prescribed 1 DOSAGE capsule FORM of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG at night FREQUENCY , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG 1000 mg STRENGTH two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

In [0]:
# ner_greedy

visualiser_greedy = NerVisualizer()

ner_greedy_vis = visualiser_greedy.display(posology_light_result[0], label_col='ner_chunk_greedy', document_col='document', return_html=True)

displayHTML(ner_greedy_vis)

The patient was prescribed 1 capsule of Advil DRUG for 5 days DURATION . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine DRUG at night FREQUENCY , 12 units of insulin lispro DRUG with meals FREQUENCY , and metformin 1000 mg DRUG two times a day FREQUENCY . It was determined that all SGLT2 inhibitors DRUG should be discontinued indefinitely for 3 months DURATION .

## Writing a generic NER function

**Generic NER Function with LightPipeline**

In [0]:
def get_light_model(embeddings, model_name = 'ner_clinical'):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

  ner_converter = NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

  nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      word_embeddings,
      loaded_ner_model,
      ner_converter])

  model = nlpPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

  return LightPipeline(model)

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

light_model = get_light_model(embeddings, model_name)

text = "I had a headache yesterday and took an Advil."

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_clinical download started this may take some time.
[ | ][ / ][OK!]

In [0]:
cols = [
     'entities_ner_chunk_origin_sentence',
     'entities_ner_chunk_begin',
     'entities_ner_chunk_end',
     'entities_ner_chunk',
     'entities_ner_chunk_class',
]

nlu.to_pretty_df(light_model, text, positions=True, output_level='chunk')[cols]

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[55]:

,entities_ner_chunk_origin_sentence,entities_ner_chunk_begin,entities_ner_chunk_end,entities_ner_chunk,entities_ner_chunk_class
0,0,6,15,a headache,PROBLEM
0,0,36,43,an Advil,TREATMENT


In [0]:
text ='''The patient was prescribed 1 capsule of Parol with meals . 
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . 
It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'''

embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

light_model = get_light_model(embeddings, model_name)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_posology download started this may take some time.
[ | ][OK!]

In [0]:
nlu.to_pretty_df(light_model, text,positions=True, output_level='chunk')[cols]

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[57]:

,entities_ner_chunk_origin_sentence,entities_ner_chunk_begin,entities_ner_chunk_end,entities_ner_chunk,entities_ner_chunk_class
0,0,27,27,1,DOSAGE
0,0,29,35,capsule,FORM
0,0,40,44,Parol,DRUG
0,0,46,55,with meals,FREQUENCY
0,1,127,134,40 units,DOSAGE
0,1,139,154,insulin glargine,DRUG
0,1,156,163,at night,FREQUENCY
0,1,167,174,12 units,DOSAGE
0,1,179,192,insulin lispro,DRUG
0,1,194,203,with meals,FREQUENCY


## PHI NER

**Entities**
- AGE
- CONTACT
- DATE
- ID
- LOCATION
- NAME
- PROFESSION

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_deid_subentity_augmented'

# deidentify_dl
# ner_deid_large
# ner_deid_generic_augmented
# ner_deid_subentity_augmented
# ner_deid_subentity_augmented_i2b2

text = """Miriam BRAY is a 41-year-old female from Vietnam and she was admitted for a right-sided pleural effusion for thoracentesis on Monday by Dr. X. Her Coumadin was placed on hold.
She was instructed to followup with Dr. XYZ in the office to check her INR On August 24, 2007 ."""

light_model = get_light_model(embeddings, model_name)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_deid_subentity_augmented download started this may take some time.
[ | ][ / ][OK!]

In [0]:
nlu.to_pretty_df(light_model,text,positions=True,output_level='chunk')[cols]

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[59]:

,entities_ner_chunk_origin_sentence,entities_ner_chunk_begin,entities_ner_chunk_end,entities_ner_chunk,entities_ner_chunk_class
0,0,0,10,Miriam BRAY,PATIENT
0,0,17,27,41-year-old,AGE
0,0,41,47,Vietnam,COUNTRY
0,0,126,131,Monday,DATE
0,0,140,140,X,DOCTOR
0,1,216,218,XYZ,DOCTOR
0,1,254,268,"August 24, 2007",DATE


## Oncology NER

There are several Oncology NER models in Spark NLP. We will show an example of `new_oncology_wip` model here.

**Entities**

| | | | |
|-|-|-|-|
| Invasion | Dosage | Hormonal_Therapy | Site_Lung |
| Biomarker_Result | Response_To_Treatment | Oncogene | Relative_Date |
| Site_Bone | Histological_Type | Radiation_Dose | Age |
| Duration | Grade | Tumor_Size | Line_Of_Therapy |
| Site_Breast | Cancer_Surgery | Smoking_Status | Imaging_Test |
| Adenopathy | Targeted_Therapy | Radiotherapy | Metastasis |
| Date | Cycle_Number | Pathology_Result | Cancer_Score |
| Cancer_Dx | Death_Entity | Site_Liver | Site_Brain |
| Pathology_Test | Chemotherapy | Biomarker | Frequency |
| Site_Lymph_Node | Performance_Status | Unspecific_Therapy | Cycle_Count |
| Race_Ethnicity | Gender | Staging | Immunotherapy |
| Site_Other_Body_Part | Tumor_Finding | Route | Cycle_Day |
| Direction| | | |

You can check [Oncology Model Notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/27.Oncology_Model.ipynb) for more examples.

In [0]:
mt_samples_df.limit(1).collect()[0]["text"]

Out[60]: 'Sample Type / Medical Specialty:\nHematology - Oncology\nSample Name:\nDischarge Summary - Mesothelioma - 1\nDescription:\nMesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.\n(Medical Transcription Sample Report)\nPRINCIPAL DIAGNOSIS:\nMesothelioma.\nSECONDARY DIAGNOSES:\nPleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.\nPROCEDURES\n1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.\n2. On August 20, 2007, thoracentesis.\n3. On August 31, 2007, Port-A-Cath placement.\nHISTORY AND PHYSICAL:\nThe patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.\nPAST MEDICAL HISTORY\n1. Pericardectomy.\n2. Pericarditis.\n2. Atrial fibrillation.\n4. RNCA with intracranial thrombolytic treatment.\n5 PTA of MCA.\n6. Mesenteric venous thrombosis.\n7. Pericardial window.\n8. Cholecystectomy.\n9. Left thoracentesis.\nFAMILY HISTORY:\nNo family history of coronary artery disease, CVA, diabetes, CHF or MI. The patient has one family member, a sister, with history of cancer.\nSOCIAL HISTORY:\nShe is married. Employed with the US Post Office. She is a mother of three. Denies tobacco, alcohol or illicit drug use.\nMEDICATIONS\n1. Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and her INR was 2.3.\n2. Amiodarone 100 mg p.o. daily.\nREVIEW OF SYSTEMS:\nComplete review of systems negative except as in pulmonary as noted above. The patient also reports occasional numbness and tingling of her left arm.\nPHYSICAL EXAMINATION\nVITAL SIGNS: Blood pressure 123/95, heart rate 83, respirations 20, temperature 97, and oxygen saturation 97%.\nGENERAL: Positive nonproductive cough and pain with coughing.\nHEENT: Pupils are equal and reactive to light and accommodation. Tympanic membranes are clear.\nNECK: Supple. No lymphadenopathy. No masses.\nRESPIRATORY: Pleural friction rub is noted.\nGI: Soft, nondistended, and nontender. Positive bowel sounds. No organomegaly.\nEXTREMITIES: No edema, no clubbing, no cyanosis, no tenderness. Full range of motion. Normal pulses in all extremities.\nSKIN: No breakdown or lesions. No ulcers.\nNEUROLOGIC: Grossly intact. No focal deficits. Awake, alert, and oriented to person, place, and time.\nLABORATORY DATA:\nLabs are pending.\nHOSPITAL COURSE:\nThe patient was admitted for a right-sided pleural effusion for thoracentesis on Monday by Dr. X. Her Coumadin was placed on hold. A repeat echocardiogram was checked. She was started on prophylaxis for DVT with Lovenox 40 mg subcutaneously. Her history dated back to March 2005 when she first sought medical attention for evidence of pericarditis, which was treated with pericardial window in an outside hospital, at that time she was also found to have mesenteric pain and thrombosis, is now anticoagulated. Her pericardial fluid was accumulated and she was seen by Dr. Y. At that time, she was recommended for pericardectomy, which was performed by Dr. Z. Review of her CT scan from March 2006 prior to her pericardectomy, already shows bilateral plural effusions. The patient improved clinically after the pericardectomy with resolution of her symptoms. Recently, she was readmitted to the hospital with chest pain and found to have bilateral pleural effusion, the right greater than the left. CT of the chest also revealed a large mediastinal lymph node. We reviewed the pathology obtained from the pericardectomy in March 2006, which was diagnostic of mesothelioma. At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid bio

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_oncology'

text =  mt_samples_df.limit(1).collect()[0]["text"]

light_model = get_light_model(embeddings, model_name)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_oncology download started this may take some time.
[ | ][ / ][OK!]

In [0]:
nlu.to_pretty_df(light_model, text, positions=True, output_level='chunk')[cols].head(20).reset_index(drop=True)

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[62]:

,entities_ner_chunk_origin_sentence,entities_ner_chunk_begin,entities_ner_chunk_end,entities_ner_chunk,entities_ner_chunk_class
0,0,118,129,Mesothelioma,Cancer_Dx
1,4,471,485,"August 24, 2007",Date
2,4,488,500,decortication,Cancer_Surgery
3,4,509,512,lung,Site_Lung
4,4,519,525,pleural,Site_Lung
5,4,527,532,biopsy,Pathology_Test
6,4,538,561,transpleural fluoroscopy,Imaging_Test
7,5,570,584,"August 20, 2007",Date
8,6,608,622,"August 31, 2007",Date
9,7,687,697,41-year-old,Gender


## NER Chunker
We can extract phrases that fits into a known pattern using the NER tags. NerChunker would be quite handy to extract entity groups with neighboring tokens when there is no pretrained NER model to address certain issues. Lets say we want to extract drug and frequency together as a single chunk even if there are some unwanted tokens between them.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_chunker = NerChunker()\
    .setInputCols(["sentence","ner"])\
    .setOutputCol("ner_chunk")\
    .setRegexParsers(["<DRUG>.*<FREQUENCY>"])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_chunker])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_chunker_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
[ | ][OK!]

In [0]:
posology_ner.getClasses()

Out[64]: ['O',
 'B-DOSAGE',
 'B-STRENGTH',
 'I-STRENGTH',
 'B-ROUTE',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'B-DRUG',
 'I-DRUG',
 'B-FORM',
 'I-DOSAGE',
 'B-DURATION',
 'I-DURATION',
 'I-FORM',
 'I-ROUTE']

In [0]:
light_model = LightPipeline(ner_chunker_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

list(zip(light_result['token'], light_result['ner']))

Out[65]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('fro', 'O'),
 ('3', 'O'),
 ('months', 'O'),
 ('.', 'O')]

In [0]:
light_result["ner_chunk"]

Out[66]: ['insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day']

## Chunk Filterer
ChunkFilterer will allow you to filter out named entities by some conditions or predefined look-up lists, so that you can feed these entities to other annotators like Assertion Status or Entity Resolvers. It can be used with two criteria: isin and regex.

In [0]:
posology_ner = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")
      
chunk_filterer = ChunkFilterer()\
    .setInputCols("sentence","ner_chunk")\
    .setOutputCol("chunk_filtered")\
    .setCriteria("isin")\
    .setWhiteList(['Advil','metformin', 'insulin lispro'])

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_posology download started this may take some time.
[ | ][OK!]

In [0]:
light_model = LightPipeline(chunk_filter_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

light_result.keys()

Out[68]: dict_keys(['document', 'ner_chunk', 'chunk_filtered', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
light_result['ner_chunk'] 

Out[69]: ['1',
 'capsule',
 'Advil',
 'for 5 days',
 '40 units',
 'insulin glargine',
 'at night',
 '12 units',
 'insulin lispro',
 'with meals',
 'metformin',
 '1000 mg',
 'two times a day',
 'SGLT2 inhibitors']

In [0]:
light_result["chunk_filtered"]

Out[70]: ['Advil', 'insulin lispro', 'metformin']

In [0]:
ner_model = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentence","token","embeddings")\
    .setOutputCol("ner")

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")
    
chunk_filterer = ChunkFilterer()\
    .setInputCols("sentence","ner_chunk")\
    .setOutputCol("chunk_filtered")\
    .setCriteria("isin")\
    .setWhiteList(['severe fever','sore throat'])

#   .setCriteria("regex")\

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ner_model,
    ner_converter,
    chunk_filterer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chunk_filter_model = nlpPipeline.fit(empty_data)

ner_clinical download started this may take some time.
[ | ][OK!]

In [0]:
text = 'Patient with severe fever, severe cough, sore throat, stomach pain, and a headache.'

filter_df = spark.createDataFrame([[text]]).toDF("text")

chunk_filter_result = chunk_filter_model.transform(filter_df)

In [0]:
chunk_filter_result.select('ner_chunk.result','chunk_filtered.result').show(truncate=False)

+-------------------------------------------------------------------+---------------------------+
result |result |
+-------------------------------------------------------------------+---------------------------+
[severe fever, severe cough, sore throat, stomach pain, a headache]|[severe fever, sore throat]|
+-------------------------------------------------------------------+---------------------------+

## Changing entity labels with `NerConverterInternal()`

There are two parameters in `NerConverterInternal()` annotator for replacing the labels. 

+ `.setReplaceLabels`: You can just create a dictionary with the labels and their new values in this parameter.

```python
jsl_ner_converter_internal = medical.NerConverterInternal()\
    .setInputCols(["sentence","token","jsl_ner"])\
    .setOutputCol("replaced_ner_chunk")\
    .setReplaceLabels({"Drug_BrandName": "Drug",
                       "Frequency": "Drug_Frequency",
                       "Dosage": "Drug_Dosage",
                       "Strength": "Drug_Strength"})
```

+ `.setReplaceDictResource`: You can feed a file that contains the labels and their new values in a dictionary format. 

```python
replace_dict = """Drug_BrandName,Drug
Frequency,Drug_Frequency
Dosage,Drug_Dosage
Strength,Drug_Strength
"""
with open('replace_dict.csv', 'w') as f:
    f.write(replace_dict)

...
jsl_ner_converter_internal = medical.NerConverterInternal()\
    .setInputCols(["sentence","token","jsl_ner"])\
    .setOutputCol("replaced_ner_chunk")\
    .setReplaceDictResource("replace_dict.csv","text", {"delimiter":","})
...
```



You can check [NerConverterInternal Annotator Page](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#nerconverterinternal) for more details.

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") 

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

jsl_ner = MedicalNerModel.pretrained("ner_jsl", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("jsl_ner")

jsl_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "jsl_ner"]) \
    .setOutputCol("jsl_ner_chunk")

jsl_ner_converter_internal = NerConverterInternal()\
    .setInputCols(["sentence","token","jsl_ner"])\
    .setOutputCol("replaced_ner_chunk")\
    .setReplaceLabels({"Drug_BrandName": "Drug",
                       "Frequency": "Drug_Frequency",
                       "Dosage": "Drug_Dosage",
                       "Strength": "Drug_Strength"})
      
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter,
    jsl_ner_converter_internal
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
ner_converter_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_jsl download started this may take some time.
[ | ][OK!]

In [0]:
text ='The patient was prescribed 1 capsule of Parol with meals. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

#light_model = LightPipeline(ner_converter_model)

cols = [
     'entities_jsl_ner_chunk_origin_sentence',
     'entities_jsl_ner_chunk_begin',
     'entities_jsl_ner_chunk_end',
     'entities_jsl_ner_chunk',
     'entities_jsl_ner_chunk_class',
     'entities_replaced_ner_chunk_class'
]
nlu.to_pretty_df(ner_converter_model,text,positions=True,output_level='chunk')[cols].head(20)

Warning::Spark Session already created, some configs may not take.
Warning::Spark Session already created, some configs may not take.
Out[75]:

,entities_jsl_ner_chunk_origin_sentence,entities_jsl_ner_chunk_begin,entities_jsl_ner_chunk_end,entities_jsl_ner_chunk,entities_jsl_ner_chunk_class,entities_replaced_ner_chunk_class
0,0,27,35,1 capsule,Dosage,Drug_Dosage
0,0,40,44,Parol,Drug_BrandName,Drug
0,1,58,59,He,Gender,Gender
0,1,77,97,endocrinology service,Clinical_Dept,Clinical_Dept
0,1,103,105,she,Gender,Gender
0,1,111,120,discharged,Admission_Discharge,Admission_Discharge
0,1,125,132,40 units,Dosage,Drug_Dosage
0,1,137,152,insulin glargine,Drug_Ingredient,Drug_Ingredient
0,1,154,161,at night,Frequency,Drug_Frequency
0,1,165,172,12 units,Dosage,Drug_Dosage


### Adding `setIgnoreStopWords` parameter to control chunk generation.
By providing `setIgnoreStopWords`, we can ignore certain words or characters while creating a chunk. <br/>

It should be a list of tokens/words or characters, and when two entities of the same type are separated by those words, these entities can be combined to produce a single, larger chunk.



Firstly let's create `NerConverterInternal()` without this parameter

In [0]:
jsl_ner= MedicalNerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models")\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setOutputCol("ner_deid")\
      .setLabelCasing('upper')

jsl_ner_converter_internal= NerConverterInternal()\
        .setInputCols(["sentence", "token", "ner_deid"])\
        .setOutputCol("chunk_deid")\
        .setGreedyMode(True)\
        .setWhiteList(['LOCATION'])
      
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter_internal
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
ner_converter_model = nlpPipeline.fit(empty_data)

ner_deid_generic_augmented download started this may take some time.
[ | ][ / ][OK!]

In [0]:
from sparknlp_display import NerVisualizer

text = """
The qualified manufacturers for this starting material are:
Alpha Chemicals Pvt LTD 
17, R K Industry House, Walbhat Rd, Goregaon – 400063 
Mumbai, Maharashtra, India
Beta Chemical Co., Ltd
Huan Cheng Xi Lu 3111hao Hai Guan Da Ting 
Shanghai, China
"""

lmodel= LightPipeline(ner_converter_model)
res = lmodel.fullAnnotate(text)[0]

ner_vis = NerVisualizer().display(res, label_col='chunk_deid', document_col='document', return_html=True)

displayHTML(ner_vis)

The qualified manufacturers for this starting material are: Alpha Chemicals Pvt LTD 17, R K Industry House LOCATION , Walbhat LOCATION Rd, Goregaon – 400063 Mumbai LOCATION , Maharashtra LOCATION , India LOCATION Beta Chemical Co., Ltd Huan Cheng Xi Lu 3111hao LOCATION Hai Guan Da Ting Shanghai LOCATION , China LOCATION

Now, let's define some characters and words with the `setIgnoreStopWords()` parameter and see the difference between the chunks.

In [0]:
jsl_ner_converter_internal= NerConverterInternal()\
        .setInputCols(["sentence", "token", "ner_deid"])\
        .setOutputCol("chunk_deid")\
        .setGreedyMode(True)\
        .setWhiteList(['LOCATION'])\
        .setIgnoreStopWords(['\n', ',', "and", 'or', '.'])
      
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    jsl_ner,
    jsl_ner_converter_internal
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
ner_converter_model = nlpPipeline.fit(empty_data)

In [0]:
lmodel= LightPipeline(ner_converter_model)
res = lmodel.fullAnnotate(text)[0]

ner_vis = NerVisualizer().display(res, label_col='chunk_deid', document_col='document', return_html=True)

displayHTML(ner_vis)

The qualified manufacturers for this starting material are: Alpha Chemicals Pvt LTD 17, R K Industry House, Walbhat LOCATION Rd, Goregaon – 400063 Mumbai, Maharashtra, India LOCATION Beta Chemical Co., Ltd Huan Cheng Xi Lu 3111hao LOCATION Hai Guan Da Ting Shanghai, China LOCATION

## Downloading Pretrained Models

- When we use `.pretrained` method, model is downloaded to  a folder named `cache_pretrained` automatically and it is loaded from this folder if you run it again.

- In order to download the models manually to any folder, you can use `ResourceDownloader.downloadModelDirectly` from `sparknlp.pretrained` or AWS CLI (steps below). In this case you should use `.load()` method.

  - Install AWS CLI to your local computer following the steps [here](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html) for Linux and [here](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-mac.html) for MacOS.

  - Then configure your AWS credentials.

  - Go to models hub and look for the model you need.

  - Select the model you found and you will see the model card that shows all the details about that model.

  - Hover the Download button on that page and you will see the download link from the S3 bucket.

In [0]:
from sparknlp.pretrained import ResourceDownloader

#The first argument is the path to the zip file and the second one is the folder.
ResourceDownloader.downloadModelDirectly("clinical/models/ner_jsl_en_3.1.0_2.4_1624566960534.zip", "cache_pretrained")  

#or you can use the classic AWS CLI 
# !aws s3 cp --region us-east-2 s3://auxdata.johnsnowlabs.com/clinical/models/ner_jsl_en_3.1.0_2.4_1624566960534.zip .

## Training a Clinical NER (NCBI Disease Dataset)

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt
  
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltest.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltrain.txt", "dbfs:/")

Out[81]: True

In [0]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, 'file:/databricks/driver/NER_NCBIconlltrain.txt')

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
conll_data.count()

Out[83]: 3266

In [0]:
from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result, conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)

+------------+-----+
ground_truth|count|
+------------+-----+
O |75093|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
conll_data.select("label.result").distinct().count()

Out[85]: 1537

In [0]:
'''
As you can see, there are too many `O` labels in the dataset. 
To make it more balanced, we can drop the sentences have only O labels.
(`c>1` means we drop all the sentences that have no valuable labels other than `O`)
'''

'''
conll_data = conll_data.withColumn('unique', F.array_distinct("label.result"))\
                       .withColumn('c', F.size('unique'))\
                       .filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)
'''

Out[86]: '\nconll_data = conll_data.withColumn(\'unique\', F.array_distinct("label.result")) .withColumn(\'c\', F.size(\'unique\')) .filter(F.col(\'c\')>1)\n\nconll_data.select(F.explode(F.arrays_zip(\'token.result\',\'label.result\')).alias("cols")) .select(F.expr("cols[\'0\']").alias("token"),\n F.expr("cols[\'1\']").alias("ground_truth")) .groupBy(\'ground_truth\') .count() .orderBy(\'count\', ascending=False) .show(100,truncate=False)\n'

In [0]:
# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
test_data = CoNLL().readDataset(spark, "file:/databricks/driver/NER_NCBIconlltest.txt")

test_data = clinical_embeddings.transform(test_data)

test_data.write.mode("overwrite").parquet('dbfs/NER_NCBIconlltest.parquet')

### NERDL Graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the `MedicalNerApproach` annotator.

In [0]:
#!pip install -q tensorflow==2.7.0
#!pip install -q tensorflow-addons

In [0]:
from sparknlp_jsl.annotator import TFGraphBuilder

Firstly, we will create graph and log folder.

In [0]:
%fs mkdirs file:/dbfs/ner/ner_logs

res0: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/ner/medical_ner_graphs

res1: Boolean = true

In [0]:
graph_folder_path = "/dbfs/ner/medical_ner_graphs"

ner_graph_builder = TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(20)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

We created ner log and graph files. 
Now, we will create graph and fit the model.

In [0]:
# TensorFlow graph file (`.pb` extension) can be produced for NER training externally. 
# If this method is used, graph folder should be added to MedicalNerApproach training 
# model as `.setGraphFolder(graph_folder_path)` .

'''
tf_graph.print_model_params("ner_dl")
tf_graph.build("ner_dl", 
               build_params={"embeddings_dim": 200, 
                             "nchars": 85, 
                             "ntags": 12, 
                             "is_medical": 1},
               model_location="/dbfs/ner/medical_ner_graphs", 
               model_filename="auto")
'''

Out[92]: '\ntf_graph.print_model_params("ner_dl")\ntf_graph.build("ner_dl", \n build_params={"embeddings_dim": 200, \n "nchars": 85, \n "ntags": 12, \n "is_medical": 1},\n model_location="/dbfs/ner/medical_ner_graphs", \n model_filename="auto")\n'

In [0]:
# for open source users
'''
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py

!pip -q install tensorflow==1.15.0

import create_graph

ntags = 3 # number of labels
embeddings_dim = 200
nchars =83

create_graph.create_graph(ntags, embeddings_dim, nchars)
'''

Out[93]: '\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py\n\n!pip -q install tensorflow==1.15.0\n\nimport create_graph\n\nntags = 3 # number of labels\nembeddings_dim = 200\nnchars =83\n\ncreate_graph.create_graph(ntags, embeddings_dim, nchars)\n'

We will use `setEarlyStoppingCriterion` and `setEarlyStoppingPatience` parameters to stop training if it doesn't improve any more. Also, these parameters will provide us to save time and resources.

- `setEarlyStoppingCriterion(float)`: This parameter is used to set the minimal improvement of the test metric to terminate training. The metric monitored is the same which is used to save the best best model (macro F1 if using test or validation set, loss otherwise). If it is F1, then training is stopped if `F1_current - F1_previous < earlyStoppingCriterion`. If monitoring loss, then training is stopped if `loss_previous - loss_current < earlyStoppingCriterion`. If earlyStoppingCriterion is `0`, then no early stopping is applied. Default is `0`.

- `setEarlyStoppingPatience(int)`: This is the number of epoch without improvement which will be tolerated. Default is `0`, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch (taking earlyStoppingCriterion into account).

In [0]:
nerTagger = MedicalNerApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(30)\
  .setBatchSize(64)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setTestDataset("/dbfs/NER_NCBIconlltest.parquet")\
  .setOutputLogsPath('dbfs:/ner/ner_logs')\
  .setUseBestModel(True)\
  .setEarlyStoppingCriterion(0.04)\
  .setEarlyStoppingPatience(3)\
  .setGraphFolder('dbfs:/ner/medical_ner_graphs')
  # .setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch       

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          ner_graph_builder,
          nerTagger
 ])

You can visit [1.4.Resume_MedicalNer_Model_Training.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb) notebook for fine-tuning pretrained NER models and more details of `MedicalNerApproach()` parameters.

In [0]:
# 20 epochs 3 min
ner_model = ner_pipeline.fit(conll_data)

# if you get an error for incompatible TF graph, use 4.1 NerDL-Graph.ipynb notebook in public folder to create a graph
# licensed users can also use 17.Graph_builder_for_DL_models.ipynb to create tf graphs easily.

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: /dbfs/ner/medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 20}
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:93: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:93: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.8/site-packages/tensorflow/python/ops/init_ops.py:93: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
ner_dl graph exported to /dbfs/ner/medical_ner_graphs/blstm_3_200_20_85.pb

If you get an error for incompatible TF graph, use [4.1 NerDL-Graph](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/4.1_NerDL_Graph.ipynb) notebook in public folder to create a graph

Licensed users can also use [17.Graph_builder_for_DL_models.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/17.Graph_builder_for_DL_models.ipynb) to create TF graphs easily.

`getTrainingClassDistribution()` parameter returns the distribution of labels used when training the NER model.

In [0]:
ner_model.stages[-1].getTrainingClassDistribution()

Out[96]: {'O': 75093, 'B-Disease': 3093, 'I-Disease': 3547}

Let's check the results saved in the log file.

In [0]:
import os 
log_file= os.listdir("/dbfs/ner/ner_logs")[-1]

with open (f"/dbfs/ner/ner_logs/{log_file}") as f:
  print(f.read())

Name of the selected graph: /local_disk0/tmp/sparknlp_tmp_721390865245131402/blstm_15_200_20_83.pb
Training started - total epochs: 21 - lr: 0.001 - batch size: 32 - labels: 15 - chars: 82 - training examples: 10468


Epoch 1/21 started, lr: 0.001, dataset size: 10468


Epoch 1/21 - 17.44s - loss: 1762.8289 - avg training loss: 6.677382 - batches: 264
Quality on validation dataset (20.0%), validation examples = 2093
time to finish evaluation: 1.51s
Total validation loss: 195.3152	Avg validation loss: 2.8723
label tp fp fn prec rec f1
B-Social_Exclusion 0 0 130 0.0 0.0 0.0
B-Substance_Use 0 0 134 0.0 0.0 0.0
I-Financial_Status 1 0 167 1.0 0.005952381 0.01183432
I-Social_Exclusion 1 0 91 1.0 0.010869565 0.021505376
B-Insurance_Status 2 0 128 1.0 0.015384615 0.03030303
B-Housing 3 0 196 1.0 0.015075377 0.029702973
B-Violence_Or_Abuse 2 1 84 0.6666667 0.023255814 0.044943817
I-Substance_Use 1 7 103 0.125 0.009615385 0.017857142
I-Violence_Or_Abuse 0 0 37 0.0 0.0 0.0
B-Financial_Status 115 9 44 0.92741936 0.7232704 0.81272084
I-Housing 12 2 164 0.85714287 0.06818182 0.12631579
I-Spiritual_Beliefs 0 0 60 0.0 0.0 0.0
B-Spiritual_Beliefs 0 0 85 0.0 0.0 0.0
I-Insurance_Status 28 22 59 0.56 0.3218391 0.40875912
tp: 165 fp: 41 fn: 1482 labels: 14
Macro-average prec: 0.50973064, rec: 0.085246034, f1: 0.14606461
Micro-average prec: 0.80097085, rec: 0.100182146, f1: 0.17808957


Epoch 2/21 started, lr: 9.950249E-4, dataset size: 10468


Epoch 2/21 - 15.62s - loss: 728.3495 - avg training loss: 2.7588997 - batches: 264
Quality on validation dataset (20.0%), validation examples = 2093
time to finish evaluation: 1.01s
Total validation loss: 121.9430	Avg validation loss: 1.7933
label tp fp fn prec rec f1
B-Social_Exclusion 35 2 95 0.9459459 0.26923078 0.4191617
B-Substance_Use 32 2 102 0.9411765 0.23880596 0.38095236
I-Financial_Status 107 14 61 0.88429755 0.6369048 0.7404845
I-Social_Exclusion 75 12 17 0.86206895 0.8152174 0.83798873
B-Insurance_Status 17 18 113 0.4857143 0.13076924 0.20606063
B-Housing 29 4 170 0.8787879 0.14572865 0.25
B-Violence_Or_Abuse 2 4 84 0.33333334 0.023255814 0.043478258
I-Substance_Use 30 18 74 0.625 0.28846154 0.39473683
I-Violence_Or_Abuse 0 0 37 0.0 0.0 0.0
B-Financial_Status 140 61 19 0.6965174 0.8805031 0.77777773
I-Housing 44 11 132 0.8 0.25 0.38095242
I-Spiritual_Beliefs 1 0 59 1.0 0.016666668 0.032786887
B-Spiritual_Beliefs 1 0 84 1.0 0.011764706 0.023255814
I-Insurance_Status 45 8 42 0.8490566 0.51724136 0.64285713
tp: 558 fp: 154 fn: 1089 labels: 14
Macro-average prec: 0.7358499, rec: 0.30175355, f1: 0.4279965
Micro-average prec: 0.78370786, rec: 0.3387978, f1: 0.47308183


Epoch 3/21 started, lr: 9.90099E-4, dataset size: 10468


Epoch 3/21 - 15.42s - loss: 514.7274 - avg training loss: 1.949725 - batches: 264
Quality on validation dataset (20.0%), validation examples = 2093
time to finish evaluation: 0.99s
Total validation loss: 76.8381	Avg validation loss: 1.1300
label tp fp fn prec rec f1
B-Social_Exclusion 115 6 15 0.9504132 0.88461536 0.9163346
B-Substance_Use 86 19 48 0.81904763 0.64179105 0.7196652
I-Financial_Status 142 15 26 0.9044586 0.8452381 0.8738461
I-Social_Exclusion 87 9 5 0.90625 0.9456522 0.9255319
B-Insurance_Status 119 17 11 0.875 0.9153846 0.8947368
B-Housing 116 13 83 0.8992248 0.5829146 0.7073172
B-Violence_Or_Abuse 21 16 65 0.5675676 0.24418604 0.34146342
I-Substance_Use 52 36 52 0.59090906 0.5 0.5416667
I-Violence_Or_Abuse 0 0 37 0.0 0.0 0.0
B-Financial_Status 141 17 18 0.8924051 0.8867925 0.88958997
I-Housing 97 7 79 0.9326923 0.5511364 0.69285715
I-Spiritual_Beliefs 33 0 27 1.0 0.55 0.70967746
B-Spiritual_Beliefs 78 2 7 0.975 0.91764706 0.9454546
I-Insurance_Status 66 6 21 0.9166667 0.7586207 0.8301887
tp: 1153 fp: 163 fn: 494 labels: 14
Macro-average prec: 0.8021168, rec: 0.65885556, f1: 0.72346216
Micro-average prec: 0.8761398, rec: 0.7000607, f1: 0.77826536


Epoch 4/21 started, lr: 9.852217E-4, dataset size: 10468


Epoch 4/21 - 15.26s - loss: 400.71713 - avg training loss:

As you see above, our earlyStopping feature worked, trainining was terminated before 30th epoch.

### Evaluate your model

In [0]:
pred_df = ner_model.stages[-1].transform(test_data)

In [0]:
pred_df.columns

Out[99]: ['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

In [0]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
 entity| tp| fp| fn|total|precision|recall| f1|
+-------+-----+-----+-----+-----+---------+------+------+
Disease|497.0|183.0|207.0|704.0| 0.7309| 0.706|0.7182|
+-------+-----+-----+-----+-----+---------+------+------+

+------------------+
 macro|
+------------------+
0.7182080924855492|
+------------------+

None
+------------------+
 micro|
+------------------+
0.7182080924855492|
+------------------+

None

In [0]:
evaler = NerDLMetrics(mode="partial_chunk_per_token")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+------+-----+-----+------+---------+------+------+
 entity| tp| fp| fn| total|precision|recall| f1|
+-------+------+-----+-----+------+---------+------+------+
Disease|1183.0|178.0|314.0|1497.0| 0.8692|0.7902|0.8279|
+-------+------+-----+-----+------+---------+------+------+

+-----------------+
 macro|
+-----------------+
0.827851644506648|
+-----------------+

None
+-----------------+
 micro|
+-----------------+
0.827851644506648|
+-----------------+

None

`ner_utils`: This new module is used after NER training to calculate mertic chunkbase and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
from sparknlp_jsl.utils.ner_utils import get_charts, loss_plot, evaluate

metrics = evaluate( preds_df['prediction'].values, preds_df['ground_truth'].values)

loss_plot(f"./ner_logs/{log_file}")

get_charts('./ner_logs/'+log_files[0])
```

In [0]:
ner_model.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/custom_Med_NER_20e')

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

loaded_ner_model = MedicalNerModel.load("dbfs:/databricks/driver/models/custom_Med_NER_20e")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

converter = NerConverterInternal()\
    .setInputCols(["document", "token", "ner"])\
    .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        clinical_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)

from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [0]:
text = "She has a metastatic breast cancer to lung"

def get_preds(text, light_model):

    result = light_model.fullAnnotate(text)[0]

    return [(i.result, i.metadata['entity']) for i in result['ner_span']]

get_preds(text, light_model)

Out[104]: [('metastatic breast cancer', 'Disease')]

## BertForTokenClassification NER models

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

tokenClassifier = MedicalBertForTokenClassifier.pretrained("bert_token_classifier_ner_clinical", "en", "clinical/models")\
    .setInputCols("token", "sentence")\
    .setOutputCol("ner")\
    .setCaseSensitive(True)

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

pipeline =  Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    tokenClassifier,
    ner_converter
    ])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

text = """A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . 
Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . 
Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . 
Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . 
The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . 
However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . 
The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . 
The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . 
Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . 
The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . 
It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge ."""

res = model.transform(spark.createDataFrame([[text]]).toDF("text"))

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
bert_token_classifier_ner_clinical download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
from pyspark.sql import functions as F

res.select(F.explode(F.arrays_zip(res.ner_chunk.result, res.ner_chunk.begin, res.ner_chunk.end, res.ner_chunk.metadata)).alias("cols")) \
   .select(F.expr("cols['3']['sentence']").alias("sentence_id"),
           F.expr("cols['0']").alias("chunk"),
           F.expr("cols['2']").alias("end"),
           F.expr("cols['3']['entity']").alias("ner_label"))\
   .filter("ner_label!='O'")\
   .show(truncate=False)

+-----------+-----------------------------+---+---------+
sentence_id|chunk |end|ner_label|
+-----------+-----------------------------+---+---------+
0 |gestational diabetes mellitus|67 |PROBLEM |
0 |type two diabetes mellitus |153|PROBLEM |
0 |T2DM |160|PROBLEM |
0 |HTG-induced pancreatitis |209|PROBLEM |
0 |an acute hepatitis |280|PROBLEM |
0 |obesity |294|PROBLEM |
0 |a body mass index |317|TEST |
0 |BMI |323|TEST |
0 |polyuria |387|PROBLEM |
0 |polydipsia |400|PROBLEM |
0 |poor appetite |416|PROBLEM |
0 |vomiting |431|PROBLEM |
1 |amoxicillin |522|TREATMENT|
1 |a respiratory tract infection|556|PROBLEM |
2 |metformin |580|TREATMENT|
2 |glipizide |592|TREATMENT|
2 |dapagliflozin |612|TREATMENT|
2 |T2DM |621|PROBLEM |
2 |atorvastatin |638|TREATMENT|
2 |gemfibrozil |654|TREATMENT|
+-----------+-----------------------------+---+---------+
only showing top 20 rows

In [0]:
light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)

from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', return_html=True)

displayHTML(ner_vis)

A 28-year-old female with a history of gestational diabetes mellitus PROBLEM diagnosed eight years prior to presentation and subsequent type two diabetes mellitus PROBLEM ( T2DM PROBLEM ), one prior episode of HTG-induced pancreatitis PROBLEM three years prior to presentation , associated with an acute hepatitis PROBLEM , and obesity PROBLEM with a body mass index TEST ( BMI TEST ) of 33.5 kg/m2 , presented with a one-week history of polyuria PROBLEM , polydipsia PROBLEM , poor appetite PROBLEM , and vomiting PROBLEM . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin TREATMENT for a respiratory tract infection PROBLEM . She was on metformin TREATMENT , glipizide TREATMENT , and dapagliflozin TREATMENT for T2DM PROBLEM and atorvastatin TREATMENT and gemfibrozil TREATMENT for HTG PROBLEM . She had been on dapagliflozin TREATMENT for six months at the time of presentation . Physical examination TEST on presentation was significant for dry oral mucosa PROBLEM ; significantly , her abdominal examination TEST was benign with no tenderness PROBLEM , guarding PROBLEM , or rigidity PROBLEM . Pertinent laboratory findings on admission were : serum glucose TEST 111 mg/dl , bicarbonate TEST 18 mmol/l , anion gap TEST 20 , creatinine TEST 0.4 mg/dL , triglycerides TEST 508 mg/dL , total cholesterol TEST 122 mg/dL , glycated hemoglobin ( TEST HbA1c TEST ) TEST 10% , and venous pH TEST 7.27 . Serum lipase TEST was normal at 43 U/L . Serum acetone levels TEST could not be assessed as blood samples TEST kept hemolyzing TEST due to significant lipemia PROBLEM . The patient was initially admitted for starvation ketosis PROBLEM , as she reported poor oral intake PROBLEM for three days prior to admission . However , serum chemistry TEST obtained six hours after presentation revealed her glucose TEST was 186 mg/dL , the anion gap TEST was still elevated PROBLEM at 21 , serum bicarbonate TEST was 16 mmol/L , triglyceride level TEST peaked at 2050 mg/dL , and lipase TEST was 52 U/L . The β-hydroxybutyrate level TEST was obtained and found to be elevated PROBLEM at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer TREATMENT removed prior to analysis TEST due to interference from turbidity PROBLEM caused by lipemia PROBLEM again . The patient was treated with an insulin drip TREATMENT for euDKA PROBLEM and HTG PROBLEM with a reduction in the anion gap TEST to 13 and triglycerides TEST to 1400 mg/dL , within 24 hours . Her euDKA PROBLEM was thought to be precipitated by her respiratory tract infection PROBLEM in the setting of SGLT2 inhibitor use TREATMENT . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine TREATMENT at night , 12 units of insulin lispro TREATMENT with meals , and metformin TREATMENT 1000 mg two times a day . It was determined that all TREATMENT SGLT2 inhibitors TREATMENT should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .

**Training Clinical BertForTokenClassification Model**

For training own BertForTokenClassification NER model, you can check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.5.BertForTokenClassification_NER_SparkNLP_with_Transformers.ipynb)

## Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once. In Spark NLP we have two different NER profiling pipelines;

- `ner_profiling_clinical` : Returns results for clinical NER models trained with `embeddings_clinical`.
- `ner_profiling_biobert` : Returns results for clinical NER models trained with `biobert_pubmed_base_cased`.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).

<center><b>NER Profiling Clinical Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| jsl_ner_wip_clinical               | jsl_ner_wip_greedy_clinical                    | jsl_rd_ner_wip_greedy_clinical   | jsl_ner_wip_modifier_clinical |
| ner_abbreviation_clinical          | ner_ade_binary                                 | ner_ade_clinical                 | ner_anatomy          |
| ner_anatomy_coarse                  | ner_bacterial_species                          | ner_biomedical_bc2gm             | ner_biomarker        |
| ner_bionlp                          | ner_chemicals                                  | ner_chemd_clinical               | ner_chemprot_clinical|
| ner_chexpert                        | ner_clinical                                   | ner_clinical_large               | ner_clinical_trials_abstracts  |
| ner_covid_trials                    | ner_deid_augmented                             | ner_deid_enriched                | ner_deid_generic_augmented |
| ner_deid_large                      | ner_deid_sd                                    | ner_deid_sd_large                | ner_deid_subentity_augmented |
| ner_deid_subentity_augmented        | ner_deid_subentity_augmented_i2b2              | ner_diseases                     | ner_diseases_large   |
| ner_drugprot_clinical               | ner_drugs                                      | ner_drugs_greedy                 | ner_drugs_large      |
| ner_eu_clinical_case                | ner_eu_clinical_condition                       | ner_events_admission_clinical    | ner_events_clinical  |
| ner_genetic_variants                | ner_human_phenotype_gene_clinical               | ner_human_phenotype_go_clinical  | ner_jsl              |
| ner_jsl_enriched                    | ner_jsl_greedy                                 | ner_jsl_slim                     | ner_living_species   |
| ner_measurements_clinical           | ner_medmentions_coarse                          | ner_nature_nero_clinical         | ner_nihss            |
| ner_oncology                        | ner_oncology_anatomy_general                    | ner_oncology_anatomy_granular    | ner_oncology_biomarker |
| ner_oncology_demographics           | ner_oncology_diagnosis                          | ner_oncology_posology            | ner_oncology_response_to_treatment |
| ner_oncology_tnm                    | ner_oncology_therapy                            | ner_oncology_unspecific_posology | ner_oncology_wip     |
| ner_pathogen                        | ner_posology                                   | ner_posology_experimental        | ner_posology_greedy  |
| ner_posology_large                  | ner_posology_small                              | ner_posology_wip                 | ner_radiology        |
| ner_radiology_wip_clinical          | ner_risk_factors                               | ner_sdoh_access_to_healthcare_wip| ner_sdoh_community_condition_wip |
| ner_sdoh_demographics_wip           | ner_sdoh_health_behaviours_problems_wip         | ner_sdoh_income_social_status_wip| ner_sdoh_mentions    |
| ner_sdoh_social_environment_wip     | ner_sdoh_substance_usage_wip                    | ner_sdoh_wip                     | ner_supplement_clinical |
| ner_token                           | ner_vop_anatomy_wip                            | ner_vop_clinical_dept_wip        | ner_vop_demographic_wip |
| ner_vop_problem_reduced_wip         | ner_vop_problem_wip                            | ner_vop_slim_wip                 | ner_vop_temporal_wip |
| ner_vop_test_wip                    | ner_vop_treatment_wip                           | ner_vop_wip                      |                       |


<b>NER Profiling BioBert Model List</b>

| | |
|-|-|
| ner_cellular_biobert           | ner_clinical_biobert             |
| ner_diseases_biobert           | ner_anatomy_coarse_biobert       |
| ner_events_biobert             | ner_human_phenotype_gene_biobert |
| ner_bionlp_biobert             | ner_posology_large_biobert       |
| ner_jsl_greedy_biobert         | jsl_rd_ner_wip_greedy_biobert    |
| ner_jsl_biobert                | ner_posology_biobert             |
| ner_anatomy_biobert            | jsl_ner_wip_greedy_biobert       |
| ner_jsl_enriched_biobert       | ner_chemprot_biobert             |
| ner_human_phenotype_go_biobert | ner_ade_biobert                  |
| ner_deid_biobert               | ner_risk_factors_biobert         |
| ner_deid_enriched_biobert      | ner_living_species_biobert                                |


</center>

In [0]:
from sparknlp.pretrained import PretrainedPipeline

clinical_profiling_pipeline = PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

ner_profiling_clinical download started this may take some time.
Approx size to download 2.9 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
text = '''A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''


clinical_result = clinical_profiling_pipeline.fullAnnotate(text)[0]
clinical_result.keys()

Out[3]: dict_keys(['ner_vop_problem_wip_chunks', 'ner_sdoh_demographics_wip_chunks', 'ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_deid_subentity_augmented_i2b2', 'ner_posology_greedy_chunks', 'ner_vop_demographic_wip_chunks', 'ner_sdoh_community_condition_wip_chunks', 'ner_radiology_wip_clinical', 'ner_vop_slim_wip', 'ner_oncology_diagnosis_chunks', 'ner_vop_test_wip_chunks', 'ner_jsl_slim', 'ner_vop_wip', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_oncology_unspecific_posology_chunks', 'ner_vop_problem_reduced_wip', 'ner_vop_anatomy_wip', 'ner_vop_problem_reduced_wip_chunks', 'ner_eu_clinical_case_chunks', 'ner_oncology_test_chunks', 'ner_deid_synthetic', 'ner_oncology_posology_chunks', 'ner_oncology_tnm_chunks', 'ner_oncology_anatomy_general_chunks', 'ner_drugs_greedy', 'ner_abbreviation_clinical_chunks', 'ner_covid_trials_chunks', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greedy', 'ner_cellular_chunks', 'ner_vop_clinical_dept_wip', 'ner_cancer_genetics_chunks', 'ner_biomedical_bc2gm_chunks', 'ner_oncology', 'ner_jsl_greedy', 'jsl_ner_wip_modifier_clinical_chunks', 'ner_drugs_greedy_chunks', 'ner_sdoh_access_to_healthcare_wip', 'ner_deid_sd_large_chunks', 'ner_diseases_chunks', 'ner_diseases_large', 'ner_chemprot_clinical', 'ner_posology_large', 'nerdl_tumour_demo_chunks', 'ner_oncology_chunks', 'ner_deid_subentity_augmented_chunks', 'ner_jsl_enriched_chunks', 'ner_genetic_variants_chunks', 'ner_sdoh_income_social_status_wip_chunks', 'ner_chexpert', 'ner_bionlp_chunks', 'ner_eu_clinical_condition_chunks', 'ner_measurements_clinical_chunks', 'ner_diseases_large_chunks', 'ner_drugs_large', 'ner_living_species_chunks', 'ner_oncology_therapy_chunks', 'ner_clinical_large', 'ner_chemicals', 'ner_sdoh_slim_wip', 'ner_radiology_chunks', 'ner_bacterial_species', 'ner_deid_augmented_chunks', 'ner_bionlp', 'ner_drugprot_clinical', 'ner_anatomy_chunks', 'ner_deid_subentity_augmented_i2b2_chunks', 'ner_deid_large', 'jsl_ner_wip_modifier_clinical', 'ner_vop_slim_wip_chunks', 'ner_chemprot_clinical_chunks', 'ner_eu_clinical_condition', 'nerdl_tumour_demo', 'ner_posology_experimental_chunks', 'ner_cancer_genetics', 'ner_biomarker', 'ner_drugprot_clinical_chunks', 'ner_vop_problem_wip', 'ner_sdoh_mentions', 'ner_posology_small', 'ner_vop_wip_chunks', 'ner_oncology_biomarker_chunks', 'ner_drugs_chunks', 'ner_deid_sd_chunks', 'ner_chemd_clinical', 'ner_human_phenotype_gene_clinical', 'ner_ade_clinical', 'ner_human_phenotype_go_clinical', 'ner_sdoh_wip_chunks', 'ner_sdoh_social_environment_wip', 'ner_sdoh_mentions_chunks', 'ner_risk_factors', 'ner_clinical', 'ner_posology_large_chunks', 'ner_cellular', 'ner_nature_nero_clinical_chunks', 'ner_deid_large_chunks', 'ner_clinical_trials_abstracts_chunks', 'ner_deid_generic_augmented_chunks', 'ner_posology_chunks', 'ner_oncology_anatomy_general', 'ner_sdoh_social_environment_wip_chunks', 'ner_supplement_clinical_chunks', 'ner_oncology_response_to_treatment_chunks', 'ner_deid_enriched_chunks', 'ner_chemd_clinical_chunks', 'ner_deid_sd_large', 'ner_oncology_biomarker', 'jsl_ner_wip_greedy_clinical', 'ner_sdoh_slim_wip_chunks', 'ner_bacterial_species_chunks', 'ner_sdoh_health_behaviours_problems_wip_chunks', 'ner_sdoh_demographics_wip', 'ner_sdoh_community_condition_wip', 'ner_vop_anatomy_wip_chunks', 'ner_living_species', 'ner_abbreviation_clinical', 'ner_oncology_test', 'ner_vop_temporal_wip_chunks', 'ner_diseases', 'jsl_rd_ner_wip_greedy_clinical', 'ner_oncology_response_to_treatment', 'ner_drugs', 'ner_drugs_large_chunks', 'ner_vop_treatment_wip', 'ner_biomarker_chunks', 'ner_clinical_trials_abstracts', 'ner_clinical_large_chunks', 'ner_anatomy_coarse', 'ner_nature_nero_clinical', 'token', 'ner_oncology_anatomy_granular', 'ner_medmentions_coarse', 'ner_oncology_therapy', 'ner_deid_sd', 'jsl_rd_ner_wip_greedy_clinical_chunks', 'ner_oncology_demographics', 'ner_deid_generic_augmented', 'n

In [0]:
def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.end for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:
    
    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)
    
  return df

In [0]:
get_token_results(clinical_result)

Out[5]:

,sentence,begin,end,token,ner_deid_augmented,ner_deid_subentity_augmented_i2b2,ner_radiology_wip_clinical,ner_vop_slim_wip,ner_jsl_slim,ner_vop_wip,ner_vop_problem_reduced_wip,ner_vop_anatomy_wip,ner_deid_synthetic,ner_drugs_greedy,ner_events_admission_clinical,ner_posology_greedy,ner_vop_clinical_dept_wip,ner_oncology,ner_jsl_greedy,ner_sdoh_access_to_healthcare_wip,ner_diseases_large,ner_chemprot_clinical,ner_posology_large,ner_chexpert,ner_drugs_large,ner_clinical_large,ner_chemicals,ner_sdoh_slim_wip,ner_bacterial_species,ner_bionlp,ner_drugprot_clinical,ner_deid_large,jsl_ner_wip_modifier_clinical,ner_eu_clinical_condition,nerdl_tumour_demo,ner_cancer_genetics,ner_biomarker,ner_vop_problem_wip,ner_sdoh_mentions,ner_posology_small,ner_chemd_clinical,ner_human_phenotype_gene_clinical,ner_ade_clinical,ner_human_phenotype_go_clinical,ner_sdoh_social_environment_wip,ner_risk_factors,ner_clinical,ner_cellular,ner_oncology_anatomy_general,ner_deid_sd_large,...,ner_sdoh_demographics_wip,ner_sdoh_community_condition_wip,ner_living_species,ner_abbreviation_clinical,ner_oncology_test,ner_diseases,jsl_rd_ner_wip_greedy_clinical,ner_oncology_response_to_treatment,ner_drugs,ner_vop_treatment_wip,ner_clinical_trials_abstracts,ner_anatomy_coarse,ner_nature_nero_clinical,ner_oncology_anatomy_granular,ner_medmentions_coarse,ner_oncology_therapy,ner_deid_sd,ner_oncology_demographics,ner_deid_generic_augmented,ner_anatomy,ner_deid_enriched,ner_deid_subentity_augmented,ner_oncology_diagnosis,ner_sdoh_health_behaviours_problems_wip,ner_measurements_clinical,ner_ade_binary,ner_nihss,ner_pathogen,ner_vop_test_wip,ner_jsl_enriched,ner_oncology_tnm,ner_posology_experimental,jsl_ner_wip_clinical,ner_biomedical_bc2gm,ner_vop_temporal_wip,ner_oncology_wip,ner_vop_demographic_wip,ner_sdoh_income_social_status_wip,ner_jsl,ner_sdoh_substance_usage_wip,ner_events_clinical,ner_supplement_clinical,ner_oncology_unspecific_posology,ner_genetic_variants,ner_sdoh_wip,ner_radiology,ner_eu_clinical_case,ner_posology,ner_oncology_posology,ner_covid_trials
0,0,0,0,A,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-patient,O,O,O
1,0,2,12,28-year-old,O,O,O,O,B-Age,B-Age,O,O,O,O,O,O,O,B-Gender,B-Age,O,O,O,O,O,O,O,O,B-Marital_Status,O,O,O,O,B-Age,O,O,O,B-Age,O,O,O,O,O,O,O,O,O,O,O,O,O,...,B-Gender,O,O,O,O,O,B-Age,O,O,O,O,O,B-Person,O,O,O,O,B-Gender,B-AGE,O,O,B-AGE,O,O,O,O,O,O,O,B-Age,O,O,B-Age,O,O,B-Gender,B-Age,O,B-Age,O,O,O,O,O,B-Gender,O,I-patient,O,O,B-Gender
2,0,14,19,female,O,O,O,B-Gender,B-Demographics,B-Gender,O,O,O,O,O,O,O,B-Gender,B-Gender,O,O,O,O,O,O,O,O,B-Gender,O,B-Organism,O,O,B-Gender,O,O,O,B-Gender,O,O,O,O,O,O,O,O,O,O,O,O,O,...,B-Gender,O,B-HUMAN,O,O,O,B-Gender,O,O,O,O,O,I-Person,O,B-Organism_Attribute,O,O,B-Gender,O,O,O,O,O,O,O,O,O,O,O,B-Gender,O,O,B-Gender,O,O,B-Gender,B-Gender,O,B-Gender,O,O,O,O,O,B-Gender,O,I-patient,O,O,B-Gender
3,0,21,24,with,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
4,0,26,26,a,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0,409,416,appetite,O,O,I-Symptom,I-Symptom,I-Symptom,I-Symptom,I-Problem,O,O,O,I-PROBLEM,O,O,O,I-Symptom,O,O,O,O,O,O,I-PROBLEM,O,O,O,O,O,O,I-Symptom,I-clinical_condition,

**Lets plot the labelled entity counts by NER models.**

In [0]:
import plotly.graph_objects as go

result_df = get_token_results(clinical_result)

label_result_df = result_df[result_df.columns[4:]].copy()
label_count_list = [label_result_df[label_result_df[i] != "O"].shape[0] for i in label_result_df.columns]

fig = go.Figure(go.Bar(x=label_count_list, y=label_result_df.columns, orientation='h'))

fig.update_layout(
    autosize = False,
    width = 1500,
    height = 1500,
    margin = dict(l=50, r=50, b=100, t=100, pad=4),
    paper_bgcolor = "LightSteelBlue",
    title={'text': "Counts of Labelled Entities", 'y':0.98, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    titlefont=dict(size=30),
    yaxis=dict(title_text="Clinical NER Models", titlefont=dict(size=30)),
)

fig.show()

## NER Model Finder Pretrained Pipeline
`ner_model_finder`  pretrained pipeline trained with bert embeddings that can be used to find the most appropriate NER model given the entity name.

In [0]:
from sparknlp.pretrained import PretrainedPipeline
finder_pipeline = PretrainedPipeline("ner_model_finder", "en", "clinical/models")

ner_model_finder download started this may take some time.
Approx size to download 148.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
result = finder_pipeline.fullAnnotate("oncology")[0]
result.keys()

Out[8]: dict_keys(['model_names'])

From the metadata in the `model_names` column, we'll get to the top models to the given 'oncology' entity and oncology related categories.

In [0]:
df= pd.DataFrame(zip(result["model_names"][0].metadata["all_k_resolutions"].split(":::"), 
                     result["model_names"][0].metadata["all_k_results"].split(":::")), 
                 columns=["category", "top_models"])

df.head()

Out[9]:

,category,top_models
0,oncology therapy,"['ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', 'ner_jsl_enriched..."
1,clinical department,"['ner_jsl', 'jsl_rd_ner_wip_greedy_clinical', 'jsl_ner_wip_modifier_clinical', 'ner_events_clini..."
2,biomedical unit,['ner_clinical_trials_abstracts']
3,cancer genetics,['ner_cancer_genetics']
4,anatomy,"['ner_bionlp', 'ner_medmentions_coarse', 'ner_chexpert', 'ner_anatomy_coarse', 'ner_anatomy', 'n..."


End of Notebook #